##Desarrollo del Caso Practico - Herramientas para Big Data - Spark##
###Programa de Master en Data Science & Business Analytics- Modalidad Virtual###
**Estudiante Richard Douglas Grijalba** 


**PREGUNTAS CASO NEGOCIO SPARK**

### Solicitudes o Cuestiones a resolver 

**A continuacion una breve descripcion de las tareas a resolver en este problema de Data Science**

* **caso 1:**  Mostrar por pantalla el número total de clientes del mes anterior con más de un contrato con la compañía.
* **caso 2:** Generar un nuevo dataframe de facturas del mes actual que asigne un 7 % de descuento a todos los contratos
de clientes que ya existían en el mes anterior (los contratos de los clientes nuevos seguirán con el mismo
importe). Mantener la columna "importe" y crear una nueva columna "importe_dto" con el nuevo importe,
casteada a dos decimales. Mostrar, además, el resultado ordenado por los campos “id_cliente” ascendente e
importe descendente.
* **caso 3:** Por problemas de saturación en la red debido al incremento en el uso de datos por el confinamiento debido a la
COVID-19, se decide limitar el uso de datos este mes subiendo la tarifa de todas las ofertas que incluyan "datos
ilimitados" en un 15 %. Obtener un DF con las mismas columnas que el DF de facturas del mes actual y, a
mayores, la columna "importe_dto".
* **caso 4:** Crear una nueva variable "grupo_edad" que agrupe a los clientes, tanto del “mes_actual” como del
“mes_anterior” en cuatro rangos según su edad asignando valores del uno al cuatro según el rango al que
pertenezcan (18-25 (1), 26-40 (2), 41-65 (3), >65 (4)).
Obtener una tabla resumen que extraiga para cada para cada uno de los cuatro grupos identificados la media
de consumo de datos, SMS enviados, “minutos_movil”, “minutos_fijo”, con todos los campos casteados a dos
decimales y ordenar el DF por “grupo_edad” ascendente. Extraer conclusiones.

* **caso 5:** Se quiere realizar un estudio por sexo para analizar si son las mujeres o los hombres quienes consumen más
datos durante el fin de semana y hacen más llamadas desde el móvil. Para ello, se deberá, sin ayuda de un
calendario, extraer el día de la semana al que corresponde cada una de las fechas del mes de agosto para
saber cuáles son fin de semana (se consideran días de fin de semana el viernes, sábado y domingo).
El DF a obtener deberá tener dos registros con las siguientes columnas: sexo, “total_mins_movil_finde”,
“total_datos_moviles_finde”. Extraer conclusiones tras presentar el DF resultante

* **caso 6:** Obtener un DF que contenga cuatro registros, que serán el cliente de cada grupo edad que más datos móviles
ha consumido durante los 15 primeros días del mes de agosto (día 15 incluido en el cálculo).
El DF deberá contener las columnas: nombre, edad, “grupo_edad”, “datos_moviles_total_15”,
“max_sms_enviados_15” (“max_sms_enviados_15” contiene el máximo de sms enviados en un día por el
cliente con más “datos_moviles” consumidos de cada “grupo_edad” durante esos 15 primeros días del mes).
Extraer conclusiones en cuanto a datos consumidos y SMS enviados por cada “grupo_edad”.

* **caso 7:** Interesa averiguar los minutos de llamadas de los clientes que son nuevos este mes para realizar un estudio del
impacto que tendrán en caso de producirse un pico de volumen de llamadas en la red.
Obtener un DF que contenga solo a los clientes nuevos de este mes con cuatro columnas:
“nombre_cliente_nuevo”, edad, “importe_total_mes_actual”, “total_minutos”.
Se ordenará el DF resultante por la columna "total_minutos", que contiene el total de minutos de llamadas fijas y
móviles durante este mes para cada cliente, ordenándolos de manera que el primer cliente nuevo que aparezca
sea el que menos minutos de llamadas ha realizado en el mes de agosto.

* **caso 8:** Obtener un DF que contenga, para los clientes que ya existían en el mes anterior y siguen dados de alta este
mes, tres columnas: nombre, edad, “n_dias_sin_sms”. La última columna se refiere a obtener el número de días
en los que el cliente no ha enviado ningún SMS durante el mes de agosto. Forzar al comando show a que
muestre 30 valores.
Si en todo el mes el cliente "A" no envió SMS tres días, esta columna deberá contener el valor “3”. Si hubiera
algún cliente que hubiese enviado al menos un SMS todos los días del mes, esta columna tendrá valor “0”).

* **caso 9:** Se desea obtener un coeficiente de ponderación que permita evaluar a cada cliente en función de su consumo
para identificar los clientes más atractivos que forman parte de la compañía.
Este cálculo solo se realizará para los clientes que tengan una sola oferta contratada con la compañía.
Este coeficiente se obtendrá en base a los consumos diarios y, por tanto, solo se tendrán en cuenta los clientes
que existen en el mes de agosto, ya que no existen datos de consumo del mes de julio. Las ponderaciones que
se darán a cada uno de los consumos son las siguientes:
0,4 --> llamadas desde telefóno móvil.
0,3 --> “datos_móviles_MB”.
0,2 --> llamadas desde teléfono fijo.
0,1 --> SMS enviados.

* **caso 10:** Se desea averiguar la fecha en la que entre los tres clientes que más datos consumen de cada “grupo_edad”
llegan a un consumo de 20 GB de datos móviles. En caso de que algún “grupo_edad” no llegue, entre los tres
clientes, a 20 GB en todo el mes, se asignará null como valor de esta columna (1 GB = 1024 MB).
Obtener un DF que contiene cuatro registros, uno para cada “grupo_edad” y tres columnas: “grupo_edad”,
“fecha_20_GB”, “datos_moviles_total_grupo_3_clientes”.
“datos_moviles_total_grupo_3_clientes” representa el total de datos consumidos en MB por los tres clientes del
grupo hasta final de mes).

Para la resolución de los ejercicios del Caso de Negocio, se partirá de las tablas iniciales que se han guardado en Hive al comienzo de la práctica resuelta

In [0]:
# Símbolo asterisco para crear comentarios que no afectan a la hora de ejecutar una celda con código PySpark
#from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName('abc').getOrCreate()

In [0]:
# Se realizan todos los imports necesarios
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import functions as sql_functions
from pyspark.sql.functions import sum as spark_sum, max as spark_max, col


In [0]:
# encoding: utf-8

### Informacion General del Problema de Negocio## 

Una compañía de telecomunicaciones quiere obtener insights de la muestra de 30 clientes obtenida al final de este mes y con información también del mes anterior. Los analistas de datos tendrán que agregar esta información, que se proporciona para responder a 10 cuestiones que ha planteado la directiva.

**Se inicia conformando, llamando, enunciando y guardando los data frames o data sets necesarios para el desarrollo del caso o problema de negocio que se nos presenta**

La informacion que tenemos para este caso o problema de negocio corresponde a : 

**Las tablas de input son las mismas que se utilizaron en el ejercicio guiado de Spark:**

* **df_clientes**: Muestra información de cada cliente (id, nombre, edad, ciudad, pais).
* **df_facturas_mes_ant**: Muestra la factura del mes anterior junto con el plan que tenían contratado para cada cliente. Un mismo cliente puede tener contratadas dos ofertas con la compañía, en cuyo caso habrá dos filas (de ahora en adelante, registros) con el mismo “id_cliente” y distinto “id_oferta”.
* **df_facturas_mes_actual**: Es una tabla equivalente a la anterior, pero muestra la información actualizada a último día del mes actual (agosto de 2020).
* **df_consumos_diarios**: Muestra para cada cliente, y diariamente, los datos móviles consumidos, el tráfico, SMS enviados y minutos de llamadas a fijos y móviles durante el mes en curso.
* **df_ofertas**: Muestra un catálogo de los distintos paquetes ofrecidos por la empresa, junto con una descripción y sus precios.


In [0]:
## Se inicia conformando, llamando, enunciando y guardando los data frames o data sets necesarios para el desarrollo del caso o problema de negocio que se nos presenta

**Vamos a tener una serie de lineas de codigo en las que inciamos llamando o creando los data frame (df) las variables o sets de datos, por lo que de inicio no estamos aun dando respuesta a los enunciados inidcados, sin embargo esta etapa es de suma importancia, en vista que sin datos o datos de calidad no se puede realizar un proyecto o solucion de Data Sciece o Business Analytics.**

In [0]:
columns = ['id_cliente','nombre','edad','sexo','provincia','pais']
data = [(1,'Pablo Perez',26,'M','Madrid','España'),(2,'Eduardo Redondo',58,'M','Bogota','Colombia'),(3,'Roberto Salazar',68,'M','Monterrey','Mexico'),
        (4,'Pedro Conde',32,'M','Madrid','España'),(5,'Ana Robles',41,'F','Valladolid','España'),(6,'David Roldan',74,'M','Guadalajara','Mexico'),
        (7,'Carmen Arauzo',19,'F','Medellin','Colombia'),(8,'Silvia Rodriguez',49,'F','Barcelona','España'),(9,'David Cardoso',24,'M','Cali','Colombia'),
        (10,'Pablo Lopez',40,'M','Sevilla','España'),(11,'Luis Rodrigues',20,'M','Bogota','Colombia'),(12,'Alvaro Monroy',75,'M','Monterrey','Mexico'),
        (13,'Victor Ruiz',53,'M','Valladolid','España'),(14,'Melisa Aguado',30,'F','Bogota','Colombia'),(15,'Cristian Cuadrado',52,'M','Guadalajara','Mexico'),
        (16,'Cristina Sanz',49,'F','Madrid','España'),(17,'Jorge Recio',18,'M','Valladolid','España'),(18,'Laura Luiz',44,'F','Cali','Colombia'),
        (19,'Juan Carlos Iglesias',38,'M','Barcelona','España'),(20,'Oscar Rico',22,'M','Valladolid','España'),(21,'Fatima Cuevas',29,'F','Cali','Colombia'),
        (22,'Clara Suarez',21,'F','Sevilla','España'),(23,'Fernanda Gomez',78,'F','Monterrey','Mexico'),(24,'Ruben Garcia',68,'M','Sevilla','España'),
        (25,'Ines Barcero',29,'F','Guadalajara','Mexico'),(26,'Celia Castro',47,'F','Barcelona','España'),(27,'Roberta Varado',64,'F','Cali','Colombia'),
        (28,'Walter Ramos',54,'M','Cali','Colombia'),(29,'Romina Verde',29,'F','Guadalajara','Mexico'),(30,'Marta Rodrigo',67,'F','Guadalajara','Mexico')]
        
        #(25,'Roberto Versero',66,'Sevilla','M','España'),(26,'Javier Ramos',36,'M','Madrid','España'),(27,'Ana Bolana',32,'F','Barcelona','España'),
        #(28,'Lucia Alonso',23,'F','Valladolid','España'),(29,'Elena Luengo',27,'F','Valencia','España'),(30,'Rebeca Velez',69,'F','Cali','Colombia'),]

df_clientes = spark.createDataFrame(data, columns)
df_clientes.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_clientes')
df_clientes.show(n=30)

+----------+--------------------+----+----+-----------+--------+
|id_cliente|              nombre|edad|sexo|  provincia|    pais|
+----------+--------------------+----+----+-----------+--------+
|         1|         Pablo Perez|  26|   M|     Madrid|  España|
|         2|     Eduardo Redondo|  58|   M|     Bogota|Colombia|
|         3|     Roberto Salazar|  68|   M|  Monterrey|  Mexico|
|         4|         Pedro Conde|  32|   M|     Madrid|  España|
|         5|          Ana Robles|  41|   F| Valladolid|  España|
|         6|        David Roldan|  74|   M|Guadalajara|  Mexico|
|         7|       Carmen Arauzo|  19|   F|   Medellin|Colombia|
|         8|    Silvia Rodriguez|  49|   F|  Barcelona|  España|
|         9|       David Cardoso|  24|   M|       Cali|Colombia|
|        10|         Pablo Lopez|  40|   M|    Sevilla|  España|
|        11|      Luis Rodrigues|  20|   M|     Bogota|Colombia|
|        12|       Alvaro Monroy|  75|   M|  Monterrey|  Mexico|
|        13|         Vict

In [0]:
display(df_clientes)  # se prueba el formato display, opcion que muestra mejor los datos del DF

id_cliente,nombre,edad,sexo,provincia,pais
1,Pablo Perez,26,M,Madrid,España
2,Eduardo Redondo,58,M,Bogota,Colombia
3,Roberto Salazar,68,M,Monterrey,Mexico
4,Pedro Conde,32,M,Madrid,España
5,Ana Robles,41,F,Valladolid,España
6,David Roldan,74,M,Guadalajara,Mexico
7,Carmen Arauzo,19,F,Medellin,Colombia
8,Silvia Rodriguez,49,F,Barcelona,España
9,David Cardoso,24,M,Cali,Colombia
10,Pablo Lopez,40,M,Sevilla,España


**DataFrame Ofertas estandar (truncate=False)**

In [0]:
# Se presentan 10 tipos de ofertas diferentes (combinaciones entre Fibra Optica, Fijo, Movil (llamadas o mensajes ilimitados), Television)

columns = ['id_oferta','descripcion','importe'] 
data = [(1,'Fibra Optica 300MB + Fijo ilimitado + Television',70.00),
        (2,'Fibra Optica 600MB + Fijo ilimitado + Television', 78.50),
        (3,'Fibra Optica 300MB + Movil llamadas y datos ilimitados + Television', 85.00),
        (4,'Fibra Optica 600MB + Movil llamadas y datos ilimitados + Television', 95.00),
        (5,'Fibra Optica 300MB + Fijo ilimitado + Movil llamadas y datos ilimitados', 102.50),
        (6,'Fibra Optica 600MB + Fijo ilimitado + Movil llamadas y datos ilimitados + Television', 124.50),
        (7,'Fibra Optica 300MB + Movil llamadas ilimitadas + Television', 80.50),
        (8,'Fibra Optica 300MB + Movil datos ilimitados + Television', 84.00),
        (9,'Fijo llamadas ilimitadas + Movil llamadas y datos ilimitados', 42.50),
        (10,'Fijo llamadas ilimitadas + Movil llamadas ilimitadas', 31.99)]

df_ofertas = spark.createDataFrame(data, columns)
df_ofertas.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_ofertas')
df_ofertas.show(n=10, truncate=False)

+---------+------------------------------------------------------------------------------------+-------+
|id_oferta|descripcion                                                                         |importe|
+---------+------------------------------------------------------------------------------------+-------+
|1        |Fibra Optica 300MB + Fijo ilimitado + Television                                    |70.0   |
|2        |Fibra Optica 600MB + Fijo ilimitado + Television                                    |78.5   |
|3        |Fibra Optica 300MB + Movil llamadas y datos ilimitados + Television                 |85.0   |
|4        |Fibra Optica 600MB + Movil llamadas y datos ilimitados + Television                 |95.0   |
|5        |Fibra Optica 300MB + Fijo ilimitado + Movil llamadas y datos ilimitados             |102.5  |
|6        |Fibra Optica 600MB + Fijo ilimitado + Movil llamadas y datos ilimitados + Television|124.5  |
|7        |Fibra Optica 300MB + Movil llamadas ilimitad

In [0]:
display(df_ofertas)

id_oferta,descripcion,importe
1,Fibra Optica 300MB + Fijo ilimitado + Television,70.0
2,Fibra Optica 600MB + Fijo ilimitado + Television,78.5
3,Fibra Optica 300MB + Movil llamadas y datos ilimitados + Television,85.0
4,Fibra Optica 600MB + Movil llamadas y datos ilimitados + Television,95.0
5,Fibra Optica 300MB + Fijo ilimitado + Movil llamadas y datos ilimitados,102.5
6,Fibra Optica 600MB + Fijo ilimitado + Movil llamadas y datos ilimitados + Television,124.5
7,Fibra Optica 300MB + Movil llamadas ilimitadas + Television,80.5
8,Fibra Optica 300MB + Movil datos ilimitados + Television,84.0
9,Fijo llamadas ilimitadas + Movil llamadas y datos ilimitados,42.5
10,Fijo llamadas ilimitadas + Movil llamadas ilimitadas,31.99


**DataFrame facturas_mes_ant**

In [0]:
columns = ['id_cliente','id_oferta','importe','fecha'] 
data = [(1,6,124.5,"2020-07-31"),(1,6,118.60,"2020-07-31"),(1,9,36.5,"2020-07-31"),
        (2,4,95.0,"2020-07-31"),(3,5,102.5,"2020-07-31"),(5,7,80.5,"2020-07-31"),
        (5,9,40.5,"2020-07-31"),(6,8,84.0,"2020-07-31"),(7,4,95.0,"2020-07-31"),
        (8,5,102.5,"2020-07-31"),(10,6,124.5,"2020-07-31"),(11,3,80.0,"2020-07-31"),
        (12,1,70.0,"2020-07-31"),(13,4,95.0,"2020-07-31"),(14,9,42.5,"2020-07-31"),
        (15,2,78.50,"2020-07-31"),(15,6,115.0,"2020-07-31"),(16,1,70.0,"2020-07-31"),
        (17,8,84.0,"2020-07-31"),(18,5,102.5,"2020-07-31"),(18,9,37.5,"2020-07-31"),
        (20,4,95.0,"2020-07-31"),(22,2,78.5,"2020-07-31"),(22,7,69.5,"2020-07-31"),
        (23,3,85.0,"2020-07-31"),(24,3,85.0,"2020-07-31"),(25,9,42.5,"2020-07-31"),
        (25,10,31.99,"2020-07-31"),(26,4,95.0,"2020-07-31"),(27,8,84.0,"2020-07-31"),
        (27,9,41.5,"2020-07-31"),(29,6,124.5,"2020-07-31"),(30,1,70.0,"2020-07-31"),
        (30,2,70.0,"2020-07-31")]
        
df_facturas_mes_ant = spark.createDataFrame(data, columns)
df_facturas_mes_ant.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_facturas_mes_ant')
df_facturas_mes_ant.show(n=40, truncate=False)

+----------+---------+-------+----------+
|id_cliente|id_oferta|importe|fecha     |
+----------+---------+-------+----------+
|1         |6        |124.5  |2020-07-31|
|1         |6        |118.6  |2020-07-31|
|1         |9        |36.5   |2020-07-31|
|2         |4        |95.0   |2020-07-31|
|3         |5        |102.5  |2020-07-31|
|5         |7        |80.5   |2020-07-31|
|5         |9        |40.5   |2020-07-31|
|6         |8        |84.0   |2020-07-31|
|7         |4        |95.0   |2020-07-31|
|8         |5        |102.5  |2020-07-31|
|10        |6        |124.5  |2020-07-31|
|11        |3        |80.0   |2020-07-31|
|12        |1        |70.0   |2020-07-31|
|13        |4        |95.0   |2020-07-31|
|14        |9        |42.5   |2020-07-31|
|15        |2        |78.5   |2020-07-31|
|15        |6        |115.0  |2020-07-31|
|16        |1        |70.0   |2020-07-31|
|17        |8        |84.0   |2020-07-31|
|18        |5        |102.5  |2020-07-31|
|18        |9        |37.5   |2020

In [0]:
display(df_facturas_mes_ant)

id_cliente,id_oferta,importe,fecha
1,6,124.5,2020-07-31
1,6,118.6,2020-07-31
1,9,36.5,2020-07-31
2,4,95.0,2020-07-31
3,5,102.5,2020-07-31
5,7,80.5,2020-07-31
5,9,40.5,2020-07-31
6,8,84.0,2020-07-31
7,4,95.0,2020-07-31
8,5,102.5,2020-07-31


**DataFrame factura_mes_actual**

In [0]:
# Se dan de baja en el mes actual: 1y,27(solo de uno de los contratos),16,20,24
# cliente 25 rebaja por fidelidad
columns = ['id_cliente','id_oferta','importe','fecha'] 
data = [(1,6,124.5,"2020-08-31"),(1,6,118.60,"2020-08-31"),(2,4,95.0,"2020-08-31"),
        (3,5,102.5,"2020-08-31"),(4,1,70.0,"2020-08-31"),(5,7,80.5,"2020-08-31"),
        (5,9,40.5,"2020-08-31"),(6,8,84.0,"2020-08-31"),(7,4,95.0,"2020-08-31"),
        (8,5,100.5,"2020-08-31"),(9,7,78.50,"2020-08-31"),(9,10,29.99,"2020-08-31"),
        (10,6,124.5,"2020-08-31"),(11,3,83.0,"2020-08-31"),(12,1,70.0,"2020-08-31"),
        (13,4,95.0,"2020-08-31"),(14,9,42.5,"2020-08-31"),(15,2,78.50,"2020-08-31"),
        (15,6,115.0,"2020-08-31"),(17,8,84.0,"2020-08-31"),(18,5,102.5,"2020-08-31"),
        (18,9,39.0,"2020-08-31"),(19,3,85.0,"2020-08-31"),(21,6,124.50,"2020-08-31"),
        (22,2,78.5,"2020-08-31"),(22,7,72.5,"2020-08-31"),(23,3,85.0,"2020-08-31"),
        (25,9,42.5,"2020-08-31"),(25,10,29.70,"2020-08-31"),(26,4,95.0,"2020-08-31"),
        (27,8,84.0,"2020-08-31"),(28,5,102.50,"2020-08-31"),(29,6,121.50,"2020-08-31"),
        (30,1,70.0,"2020-08-31"),(30,2,71.0,"2020-08-31")]

df_facturas_mes_actual = spark.createDataFrame(data, columns)

# df_facturas_mes_actual.show(n=40, truncate=False)
df_facturas_mes_actual.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_facturas_mes_actual')

In [0]:
display(df_facturas_mes_actual)

id_cliente,id_oferta,importe,fecha
1,6,124.5,2020-08-31
1,6,118.6,2020-08-31
2,4,95.0,2020-08-31
3,5,102.5,2020-08-31
4,1,70.0,2020-08-31
5,7,80.5,2020-08-31
5,9,40.5,2020-08-31
6,8,84.0,2020-08-31
7,4,95.0,2020-08-31
8,5,100.5,2020-08-31


**DataFrame consumos_diarios** 


In [0]:
columns = ['id_cliente','consumo_datos_MB','sms_enviados','minutos_llamadas_movil','minutos_llamadas_fijo','fecha']
data = [(1,664,3,25,0,"2020-08-01"),(1,943,0,12,5,"2020-08-02"),(1,1035,1,46,10,"2020-08-03"),(1,760,3,17,0,"2020-08-04"),
        (1,1409,1,31,4,"2020-08-05"),(1,1627,0,20,0,"2020-08-06"),(1,739,3,21,0,"2020-08-07"),(1,760,0,12,0,"2020-08-08"),
        (1,928,0,0,24,"2020-08-09"),(1,829,2,46,0,"2020-08-10"),(1,1810,3,0,0,"2020-08-11"),(1,760,0,25,0,"2020-08-12"),
        (1,603,0,41,0,"2020-08-13"),(1,732,0,34,102,"2020-08-14"),(1,729,1,0,50,"2020-08-15"),(1,760,1,31,0,"2020-08-16"),
        (1,1934,2,78,0,"2020-08-17"),(1,827,0,29,32,"2020-08-18"),(1,691,0,0,7,"2020-08-19"),(1,760,2,19,0,"2020-08-20"),
        (1,1142,5,14,0,"2020-08-21"),(1,1242,1,40,0,"2020-08-22"),(1,830,0,32,2,"2020-08-23"),(1,760,4,56,17,"2020-08-24"),
        (1,734,0,19,0,"2020-08-25"),(1,1729,0,11,0,"2020-08-26"),(1,819,1,95,18,"2020-08-27"),(1,760,0,60,12,"2020-08-28"),
        (1,302,0,29,0,"2020-08-29"),(1,497,3,14,0,"2020-08-30"),(1,1281,2,21,0,"2020-08-31"),
        (2,321,7,38,13,"2020-08-01"),(2,145,0,8,0,"2020-08-02"),(2,420,1,46,4,"2020-08-03"),(2,492,1,9,0,"2020-08-04"),
        (2,264,8,29,31,"2020-08-05"),(2,172,10,22,0,"2020-08-06"),(2,314,2,74,34,"2020-08-07"),(2,312,2,0,16,"2020-08-08"),
        (2,312,11,32,27,"2020-08-09"),(2,243,2,48,0,"2020-08-10"),(2,423,4,38,12,"2020-08-11"),(2,492,1,10,22,"2020-08-12"),
        (2,492,12,29,29,"2020-08-13"),(2,274,4,60,0,"2020-08-14"),(2,402,0,28,20,"2020-08-15"),(2,471,0,12,30,"2020-08-16"),
        (2,462,1,36,32,"2020-08-17"),(2,294,3,78,42,"2020-08-18"),(2,593,0,29,65,"2020-08-19"),(2,380,12,53,9,"2020-08-20"),
        (2,568,2,42,20,"2020-08-21"),(2,300,2,40,35,"2020-08-22"),(2,600,0,35,29,"2020-08-23"),(2,397,9,37,40,"2020-08-24"),
        (2,623,4,59,42,"2020-08-25"),(2,302,1,34,15,"2020-08-26"),(2,319,1,52,34,"2020-08-27"),(2,901,8,28,83,"2020-08-28"),
        (2,611,0,41,38,"2020-08-29"),(2,319,0,21,19,"2020-08-30"),(2,387,3,19,18,"2020-08-31"),
        (3,123,5,42,14,"2020-08-01"),(3,460,4,24,16,"2020-08-02"),(3,291,3,27,18,"2020-08-03"),(3,149,7,20,0,"2020-08-04"),
        (3,182,6,20,19,"2020-08-05"),(3,245,12,28,21,"2020-08-06"),(3,302,2,31,26,"2020-08-07"),(3,200,8,18,8,"2020-08-08"),
        (3,239,7,40,25,"2020-08-09"),(3,111,9,29,0,"2020-08-10"),(3,191,1,14,29,"2020-08-11"),(3,28,12,4,10,"2020-08-12"),
        (3,201,3,15,32,"2020-08-13"),(3,102,8,31,0,"2020-08-14"),(3,62,10,29,31,"2020-08-15"),(3,82,2,49,42,"2020-08-16"),
        (3,72,1,0,15,"2020-08-17"),(3,94,2,12,18,"2020-08-18"),(3,30,11,20,45,"2020-08-19"),(3,172,1,63,8,"2020-08-20"),
        (3,80,1,12,30,"2020-08-21"),(3,274,7,5,11,"2020-08-22"),(3,219,7,8,15,"2020-08-23"),(3,104,7,6,2,"2020-08-24"),
        (3,180,9,48,45,"2020-08-25"),(3,210,0,10,44,"2020-08-26"),(3,310,2,17,18,"2020-08-27"),(3,182,9,0,1,"2020-08-28"),
        (3,164,0,32,5,"2020-08-29"),(3,289,0,12,33,"2020-08-30"),(3,126,3,23,22,"2020-08-31"),
        (4,523,0,87,0,"2020-08-01"),(4,628,1,56,0,"2020-08-02"),(4,1021,0,15,0,"2020-08-03"),(4,610,1,45,0,"2020-08-04"),
        (4,612,0,65,0,"2020-08-05"),(4,359,0,102,15,"2020-08-06"),(4,832,1,62,0,"2020-08-07"),(4,583,0,30,0,"2020-08-08"),
        (4,490,2,81,20,"2020-08-09"),(4,479,0,81,0,"2020-08-10"),(4,792,2,19,0,"2020-08-11"),(4,1216,0,27,32,"2020-08-12"),
        (4,512,0,32,0,"2020-08-13"),(4,824,4,76,0,"2020-08-14"),(4,682,1,37,18,"2020-08-15"),(4,829,2,47,0,"2020-08-16"),
        (4,921,0,45,0,"2020-08-17"),(4,729,0,42,0,"2020-08-18"),(4,534,2,95,64,"2020-08-19"),(4,967,3,51,82,"2020-08-20"),
        (4,721,3,54,0,"2020-08-21"),(4,218,0,38,17,"2020-08-22"),(4,729,0,85,0,"2020-08-23"),(4,826,1,38,0,"2020-08-24"),
        (4,845,1,34,0,"2020-08-25"),(4,367,2,71,21,"2020-08-26"),(4,502,0,60,0,"2020-08-27"),(4,800,0,25,0,"2020-08-28"),
        (4,629,0,49,0,"2020-08-29"),(4,588,0,50,0,"2020-08-30"),(4,693,1,31,40,"2020-08-31"),
        (5,241,1,125,20,"2020-08-01"),(5,315,0,36,32,"2020-08-02"),(5,301,1,20,80,"2020-08-03"),(5,152,5,38,62,"2020-08-04"),
        (5,289,4,110,45,"2020-08-05"),(5,510,2,74,40,"2020-08-06"),(5,178,2,19,67,"2020-08-07"),(5,241,3,41,59,"2020-08-08"),
        (5,301,6,62,39,"2020-08-09"),(5,86,4,82,47,"2020-08-10"),(5,201,6,75,61,"2020-08-11"),(5,358,8,51,62,"2020-08-12"),
        (5,176,8,45,0,"2020-08-13"),(5,109,5,102,12,"2020-08-14"),(5,798,5,91,45,"2020-08-15"),(5,301,0,43,71,"2020-08-16"),
        (5,198,9,73,70,"2020-08-17"),(5,125,2,115,17,"2020-08-18"),(5,182,4,99,49,"2020-08-19"),(5,222,0,30,80,"2020-08-20"),
        (5,234,3,82,34,"2020-08-21"),(5,201,3,70,20,"2020-08-22"),(5,283,3,130,27,"2020-08-23"),(5,333,2,28,0,"2020-08-24"),
        (5,387,5,90,41,"2020-08-25"),(5,258,5,62,30,"2020-08-26"),(5,251,7,100,36,"2020-08-27"),(5,475,6,79,16,"2020-08-28"),
        (5,312,12,15,80,"2020-08-29"),(5,328,1,50,19,"2020-08-30"),(5,291,11,32,65,"2020-08-31"),
        (6,50,7,10,50,"2020-08-01"),(6,80,6,10,0,"2020-08-02"),(6,65,12,0,40,"2020-08-03"),(6,284,14,0,42,"2020-08-04"),
        (6,20,9,0,46,"2020-08-05"),(6,36,4,0,67,"2020-08-06"),(6,90,9,10,36,"2020-08-07"),(6,201,18,0,83,"2020-08-08"),
        (6,28,13,0,64,"2020-08-09"),(6,150,7,22,46,"2020-08-10"),(6,135,7,0,65,"2020-08-11"),(6,196,22,17,68,"2020-08-12"),
        (6,49,15,0,37,"2020-08-13"),(6,0,0,0,92,"2020-08-14"),(6,122,6,0,56,"2020-08-15"),(6,146,3,0,94,"2020-08-16"),
        (6,102,13,15,48,"2020-08-17"),(6,38,8,0,17,"2020-08-18"),(6,180,11,14,41,"2020-08-19"),(6,159,6,0,26,"2020-08-20"),
        (6,27,16,24,50,"2020-08-21"),(6,156,12,0,75,"2020-08-22"),(6,193,10,20,39,"2020-08-23"),(6,478,4,30,64,"2020-08-24"),
        (6,35,5,0,80,"2020-08-25"),(6,120,15,28,52,"2020-08-26"),(6,201,8,0,54,"2020-08-27"),(6,310,1,28,15,"2020-08-28"),
        (6,40,3,8,35,"2020-08-29"),(6,73,4,16,34,"2020-08-30"),(6,174,7,25,13,"2020-08-31"),
        (7,1456,0,37,0,"2020-08-01"),(7,946,0,24,24,"2020-08-02"),(7,1302,0,72,0,"2020-08-03"),(7,1356,0,13,45,"2020-08-04"),
        (7,1256,1,28,0,"2020-08-05"),(7,985,0,11,5,"2020-08-06"),(7,934,0,40,0,"2020-08-07"),(7,464,1,65,0,"2020-08-08"),
        (7,930,0,0,0,"2020-08-09"),(7,1385,0,30,0,"2020-08-10"),(7,635,0,38,0,"2020-08-11"),(7,588,2,60,0,"2020-08-12"),
        (7,849,2,40,0,"2020-08-13"),(7,1405,3,10,0,"2020-08-14"),(7,911,2,67,24,"2020-08-15"),(7,1495,0,83,0,"2020-08-16"),
        (7,830,0,47,23,"2020-08-17"),(7,1295,0,52,0,"2020-08-18"),(7,844,1,9,0,"2020-08-19"),(7,1246,0,12,4,"2020-08-20"),
        (7,1044,0,28,0,"2020-08-21"),(7,732,1,0,0,"2020-08-22"),(7,1468,4,13,0,"2020-08-23"),(7,967,0,79,0,"2020-08-24"),
        (7,1953,1,17,19,"2020-08-25"),(7,802,0,24,11,"2020-08-26"),(7,1853,1,71,15,"2020-08-27"),(7,1344,0,61,16,"2020-08-28"),
        (7,1067,2,13,0,"2020-08-29"),(7,1740,0,33,57,"2020-08-30"),(7,1122,0,38,12,"2020-08-31"),
        (8,346,5,75,30,"2020-08-01"),(8,502,6,74,40,"2020-08-02"),(8,482,1,12,62,"2020-08-03"),(8,200,10,63,69,"2020-08-04"),
        (8,386,6,83,29,"2020-08-05"),(8,310,3,62,28,"2020-08-06"),(8,500,3,123,0,"2020-08-07"),(8,284,5,71,31,"2020-08-08"),
        (8,305,12,42,19,"2020-08-09"),(8,375,13,80,15,"2020-08-10"),(8,319,5,84,29,"2020-08-11"),(8,249,4,42,53,"2020-08-12"),
        (8,401,10,26,0,"2020-08-13"),(8,281,15,37,76,"2020-08-14"),(8,318,7,86,34,"2020-08-15"),(8,394,8,39,12,"2020-08-16"),
        (8,250,8,56,63,"2020-08-17"),(8,272,4,42,59,"2020-08-18"),(8,492,1,70,46,"2020-08-19"),(8,365,3,51,33,"2020-08-20"),
        (8,284,2,12,25,"2020-08-21"),(8,266,5,50,15,"2020-08-22"),(8,172,1,67,12,"2020-08-23"),(8,402,0,68,8,"2020-08-24"),
        (8,265,4,103,31,"2020-08-25"),(8,388,9,62,43,"2020-08-26"),(8,193,1,48,43,"2020-08-27"),(8,95,6,77,18,"2020-08-28"),
        (8,192,0,93,22,"2020-08-29"),(8,293,12,71,55,"2020-08-30"),(8,174,1,52,36,"2020-08-31"),
        (9,1304,0,13,0,"2020-08-01"),(9,720,0,11,5,"2020-08-02"),(9,1111,1,8,12,"2020-08-03"),(9,1760,1,5,1,"2020-08-04"),
        (9,1502,1,0,3,"2020-08-05"),(9,1455,2,21,0,"2020-08-06"),(9,850,2,7,0,"2020-08-07"),(9,799,2,7,7,"2020-08-08"),
        (9,623,0,5,0,"2020-08-09"),(9,1294,1,5,0,"2020-08-10"),(9,950,0,4,0,"2020-08-11"),(9,923,0,15,0,"2020-08-12"),
        (9,730,0,8,0,"2020-08-13"),(9,823,0,0,0,"2020-08-14"),(9,957,0,0,7,"2020-08-15"),(9,829,0,0,2,"2020-08-16"),
        (9,945,0,0,5,"2020-08-17"),(9,392,0,0,3,"2020-08-18"),(9,1028,0,3,12,"2020-08-19"),(9,729,0,0,0,"2020-08-20"),
        (9,920,4,4,0,"2020-08-21"),(9,720,1,2,9,"2020-08-22"),(9,1932,0,8,27,"2020-08-23"),(9,999,3,0,6,"2020-08-24"),
        (9,869,0,8,0,"2020-08-25"),(9,1830,0,20,15,"2020-08-26"),(9,1234,1,0,0,"2020-08-27"),(9,1885,4,24,13,"2020-08-28"),
        (9,888,2,13,0,"2020-08-29"),(9,929,1,7,6,"2020-08-30"),(9,606,2,37,0,"2020-08-31"),
        (10,482,3,35,20,"2020-08-01"),(10,418,0,32,18,"2020-08-02"),(10,382,3,22,24,"2020-08-03"),(10,212,2,30,16,"2020-08-04"),
        (10,510,7,43,6,"2020-08-05"),(10,399,1,43,25,"2020-08-06"),(10,593,0,23,25,"2020-08-07"),(10,392,2,4,25,"2020-08-08"),
        (10,692,0,13,17,"2020-08-09"),(10,312,0,2,15,"2020-08-10"),(10,492,0,13,27,"2020-08-11"),(10,433,0,32,18,"2020-08-12"),
        (10,582,2,0,29,"2020-08-13"),(10,422,3,31,6,"2020-08-14"),(10,598,1,4,46,"2020-08-15"),(10,510,0,50,12,"2020-08-16"),
        (10,382,5,34,0,"2020-08-17"),(10,628,0,0,21,"2020-08-18"),(10,428,4,22,37,"2020-08-19"),(10,413,1,12,34,"2020-08-20"),
        (10,318,6,6,7,"2020-08-21"),(10,700,2,16,30,"2020-08-22"),(10,1021,5,11,32,"2020-08-23"),(10,284,8,0,28,"2020-08-24"),
        (10,492,2,3,6,"2020-08-25"),(10,238,0,21,16,"2020-08-26"),(10,453,0,0,17,"2020-08-27"),(10,333,10,6,14,"2020-08-28"),
        (10,609,0,14,16,"2020-08-29"),(10,395,5,0,7,"2020-08-30"),(10,530,1,9,0,"2020-08-31"),
        (11,2134,0,8,11,"2020-08-01"),(11,1093,0,18,12,"2020-08-02"),(11,1467,0,15,2,"2020-08-03"),(11,1760,0,4,9,"2020-08-04"),
        (11,1482,0,12,0,"2020-08-05"),(11,1004,0,0,2,"2020-08-06"),(11,2402,1,0,8,"2020-08-07"),(11,838,1,0,0,"2020-08-08"),
        (11,922,0,0,0,"2020-08-09"),(11,983,2,2,3,"2020-08-10"),(11,1803,1,7,10,"2020-08-11"),(11,927,0,12,4,"2020-08-12"),
        (11,1450,1,16,3,"2020-08-13"),(11,889,0,7,9,"2020-08-14"),(11,1234,0,28,0,"2020-08-15"),(11,1583,0,17,0,"2020-08-16"),
        (11,1503,0,0,0,"2020-08-17"),(11,798,1,0,3,"2020-08-18"),(11,1652,0,0,0,"2020-08-19"),(11,1362,0,0,6,"2020-08-20"),
        (11,1793,0,0,0,"2020-08-21"),(11,1928,0,0,0,"2020-08-22"),(11,821,0,7,11,"2020-08-23"),(11,1278,1,0,0,"2020-08-24"),
        (11,1112,0,7,18,"2020-08-25"),(11,1356,0,19,4,"2020-08-26"),(11,629,2,15,0,"2020-08-27"),(11,1029,2,8,13,"2020-08-28"),
        (11,829,0,4,8,"2020-08-29"),(11,1224,3,0,0,"2020-08-30"),(11,492,0,6,8,"2020-08-31"),
        (12,92,5,32,30,"2020-08-01"),(12,201,1,21,17,"2020-08-02"),(12,218,12,8,13,"2020-08-03"),(12,49,2,21,4,"2020-08-04"),
        (12,134,8,16,40,"2020-08-05"),(12,192,8,19,23,"2020-08-06"),(12,167,15,9,6,"2020-08-07"),(12,92,12,24,16,"2020-08-08"),
        (12,27,1,39,21,"2020-08-09"),(12,78,17,27,15,"2020-08-10"),(12,51,8,17,8,"2020-08-11"),(12,106,14,12,13,"2020-08-12"),
        (12,39,5,27,72,"2020-08-13"),(12,90,4,18,31,"2020-08-14"),(12,73,9,10,0,"2020-08-15"),(12,176,12,9,7,"2020-08-16"),
        (12,182,0,17,52,"2020-08-17"),(12,91,8,12,8,"2020-08-18"),(12,82,7,0,6,"2020-08-19"),(12,234,6,0,15,"2020-08-20"),
        (12,76,9,10,29,"2020-08-21"),(12,108,6,18,8,"2020-08-22"),(12,72,9,14,25,"2020-08-23"),(12,188,2,7,8,"2020-08-24"),
        (12,111,13,11,17,"2020-08-25"),(12,115,0,7,9,"2020-08-26"),(12,63,6,30,4,"2020-08-27"),(12,177,0,17,22,"2020-08-28"),
        (12,124,12,20,28,"2020-08-29"),(12,100,3,0,13,"2020-08-30"),(12,94,14,19,13,"2020-08-31"),  
        (13,530,5,15,12,"2020-08-01"),(13,736,4,0,0,"2020-08-02"),(13,649,4,13,22,"2020-08-03"),(13,698,6,0,0,"2020-08-04"),
        (13,721,2,17,23,"2020-08-05"),(13,816,2,18,6,"2020-08-06"),(13,701,3,19,18,"2020-08-07"),(13,726,7,0,17,"2020-08-08"),
        (13,394,3,9,18,"2020-08-09"),(13,793,0,9,2,"2020-08-10"),(13,612,9,0,5,"2020-08-11"),(13,746,2,14,8,"2020-08-12"),
        (13,455,1,0,14,"2020-08-13"),(13,439,1,21,7,"2020-08-14"),(13,682,8,7,9,"2020-08-15"),(13,893,0,19,23,"2020-08-16"),
        (13,801,0,13,0,"2020-08-17"),(13,399,7,2,15,"2020-08-18"),(13,723,0,16,15,"2020-08-19"),(13,487,0,21,22,"2020-08-20"),
        (13,739,6,20,8,"2020-08-21"),(13,465,6,3,5,"2020-08-22"),(13,788,3,7,13,"2020-08-23"),(13,589,5,12,6,"2020-08-24"),
        (13,666,2,13,3,"2020-08-25"),(13,582,0,0,16,"2020-08-26"),(13,625,7,12,12,"2020-08-27"),(13,501,3,4,8,"2020-08-28"),
        (13,615,0,9,14,"2020-08-29"),(13,611,3,4,16,"2020-08-30"),(13,791,4,9,28,"2020-08-31"),
        (14,983,1,34,61,"2020-08-01"),(14,999,1,78,32,"2020-08-02"),(14,1039,0,34,19,"2020-08-03"),(14,772,1,53,31,"2020-08-04"),
        (14,1023,0,29,27,"2020-08-05"),(14,983,0,60,31,"2020-08-06"),(14,1324,0,28,29,"2020-08-07"),(14,861,3,49,30,"2020-08-08"),
        (14,1034,0,38,42,"2020-08-09"),(14,836,0,0,46,"2020-08-10"),(14,927,0,35,44,"2020-08-11"),(14,697,2,26,8,"2020-08-12"),
        (14,802,0,55,12,"2020-08-13"),(14,782,0,13,34,"2020-08-14"),(14,915,1,50,18,"2020-08-15"),(14,810,0,20,17,"2020-08-16"),
        (14,709,2,71,22,"2020-08-17"),(14,888,2,12,19,"2020-08-18"),(14,827,0,38,33,"2020-08-19"),(14,1293,0,9,29,"2020-08-20"),
        (14,1823,0,59,0,"2020-08-21"),(14,1083,0,23,12,"2020-08-22"),(14,1381,1,29,19,"2020-08-23"),(14,1173,1,13,39,"2020-08-24"),
        (14,1432,0,24,47,"2020-08-25"),(14,1063,0,42,19,"2020-08-26"),(14,1402,0,33,59,"2020-08-27"),(14,929,0,13,74,"2020-08-28"),
        (14,1304,0,14,59,"2020-08-29"),(14,1145,3,33,30,"2020-08-30"),(14,1102,0,72,0,"2020-08-31"),
        (15,539,3,13,26,"2020-08-01"),(15,617,0,27,14,"2020-08-02"),(15,729,0,18,10,"2020-08-03"),(15,521,1,0,24,"2020-08-04"),
        (15,491,0,14,22,"2020-08-05"),(15,719,4,7,27,"2020-08-06"),(15,821,0,8,11,"2020-08-07"),(15,594,1,32,7,"2020-08-08"),
        (15,928,4,8,16,"2020-08-09"),(15,830,3,0,16,"2020-08-10"),(15,902,3,0,6,"2020-08-11"),(15,492,4,22,12,"2020-08-12"),
        (15,619,6,9,14,"2020-08-13"),(15,715,2,25,2,"2020-08-14"),(15,762,5,12,1,"2020-08-15"),(15,378,7,3,24,"2020-08-16"),
        (15,729,8,17,0,"2020-08-17"),(15,785,6,4,8,"2020-08-18"),(15,681,12,17,27,"2020-08-19"),(15,402,5,8,2,"2020-08-20"),
        (15,800,2,0,6,"2020-08-21"),(15,516,8,0,0,"2020-08-22"),(15,535,3,9,5,"2020-08-23"),(15,501,4,7,1,"2020-08-24"),
        (15,910,1,2,6,"2020-08-25"),(15,684,9,21,9,"2020-08-26"),(15,587,2,18,12,"2020-08-27"),(15,610,2,14,18,"2020-08-28"),
        (15,527,5,16,0,"2020-08-29"),(15,901,2,13,15,"2020-08-30"),(15,534,0,6,14,"2020-08-31"),
        (17,1648,0,3,0,"2020-08-01"),(17,1849,0,6,11,"2020-08-02"),(17,1453,0,12,10,"2020-08-03"),(17,921,0,13,5,"2020-08-04"),
        (17,1623,0,0,0,"2020-08-05"),(17,1632,0,0,0,"2020-08-06"),(17,1829,0,2,9,"2020-08-07"),(17,1834,1,0,5,"2020-08-08"),
        (17,2341,0,0,7,"2020-08-09"),(17,2483,2,0,8,"2020-08-10"),(17,1712,1,2,3,"2020-08-11"),(17,1749,0,12,9,"2020-08-12"),
        (17,1232,0,0,0,"2020-08-13"),(17,2013,0,2,0,"2020-08-14"),(17,1144,1,4,7,"2020-08-15"),(17,1334,2,21,0,"2020-08-16"),
        (17,1552,0,0,0,"2020-08-17"),(17,1934,0,3,5,"2020-08-18"),(17,1536,0,11,2,"2020-08-19"),(17,1346,0,3,1,"2020-08-20"),
        (17,920,1,8,6,"2020-08-21"),(17,1235,1,10,2,"2020-08-22"),(17,2184,1,13,8,"2020-08-23"),(17,1765,3,6,8,"2020-08-24"),
        (17,1928,0,5,4,"2020-08-25"),(17,1321,0,5,1,"2020-08-26"),(17,2562,0,0,18,"2020-08-27"),(17,1113,0,11,11,"2020-08-28"),
        (17,1057,0,3,0,"2020-08-29"),(17,1022,0,0,7,"2020-08-30"),(17,1240,0,0,0,"2020-08-31"),
        (18,782,0,46,54,"2020-08-01"),(18,492,6,40,31,"2020-08-02"),(18,362,1,49,28,"2020-08-03"),(18,593,7,12,34,"2020-08-04"),
        (18,892,2,24,62,"2020-08-05"),(18,584,3,4,52,"2020-08-06"),(18,392,0,21,35,"2020-08-07"),(18,793,1,29,29,"2020-08-08"),
        (18,1639,4,32,23,"2020-08-09"),(18,493,0,62,14,"2020-08-10"),(18,418,4,44,49,"2020-08-11"),(18,632,0,85,24,"2020-08-12"),
        (18,622,1,18,27,"2020-08-13"),(18,302,4,21,19,"2020-08-14"),(18,472,2,20,38,"2020-08-15"),(18,535,5,37,19,"2020-08-16"),
        (18,522,5,9,49,"2020-08-17"),(18,592,7,42,35,"2020-08-18"),(18,520,3,19,37,"2020-08-19"),(18,589,5,20,29,"2020-08-20"),
        (18,482,6,19,18,"2020-08-21"),(18,857,4,19,72,"2020-08-22"),(18,539,8,4,13,"2020-08-23"),(18,470,1,44,31,"2020-08-24"),
        (18,527,2,49,38,"2020-08-25"),(18,496,0,23,32,"2020-08-26"),(18,611,6,39,19,"2020-08-27"),(18,593,2,54,40,"2020-08-28"),
        (18,812,0,75,21,"2020-08-29"),(18,523,2,29,42,"2020-08-30"),(18,389,0,82,14,"2020-08-31"),
        (19,1345,1,18,21,"2020-08-01"),(19,1520,3,20,18,"2020-08-02"),(19,1037,4,19,12,"2020-08-03"),(19,1628,0,18,11,"2020-08-04"),
        (19,1129,0,14,7,"2020-08-05"),(19,1045,0,19,19,"2020-08-06"),(19,1122,3,14,15,"2020-08-07"),(19,1072,1,23,14,"2020-08-08"),
        (19,1249,2,19,17,"2020-08-09"),(19,1294,4,14,16,"2020-08-10"),(19,972,0,20,13,"2020-08-11"),(19,882,0,19,8,"2020-08-12"),
        (19,938,2,20,8,"2020-08-13"),(19,1183,0,9,8,"2020-08-14"),(19,927,2,28,19,"2020-08-15"),(19,926,1,10,7,"2020-08-16"),
        (19,910,0,11,16,"2020-08-17"),(19,1009,3,15,20,"2020-08-18"),(19,899,0,7,9,"2020-08-19"),(19,935,4,12,19,"2020-08-20"),
        (19,888,0,12,22,"2020-08-21"),(19,1212,2,11,18,"2020-08-22"),(19,1322,3,11,15,"2020-08-23"),(19,1088,2,6,13,"2020-08-24"),
        (19,1420,4,21,29,"2020-08-25"),(19,988,0,17,9,"2020-08-26"),(19,1129,0,8,10,"2020-08-27"),(19,1333,0,17,16,"2020-08-28"),
        (19,1333,1,27,19,"2020-08-29"),(19,892,1,10,4,"2020-08-30"),(19,1341,2,13,9,"2020-08-31"),
        (21,1483,0,53,34,"2020-08-01"),(21,1938,0,34,42,"2020-08-02"),(21,1305,0,78,22,"2020-08-03"),(21,1324,0,25,39,"2020-08-04"),
        (21,1837,0,32,28,"2020-08-05"),(21,2183,0,21,38,"2020-08-06"),(21,1735,1,26,43,"2020-08-07"),(21,1253,3,41,19,"2020-08-08"),
        (21,983,1,35,20,"2020-08-09"),(21,1634,3,17,29,"2020-08-10"),(21,1538,0,33,19,"2020-08-11"),(21,1653,0,55,67,"2020-08-12"),
        (21,1038,0,21,17,"2020-08-13"),(21,826,0,20,17,"2020-08-14"),(21,1492,1,82,34,"2020-08-15"),(21,1403,0,31,56,"2020-08-16"),
        (21,1392,0,27,19,"2020-08-17"),(21,1542,0,17,34,"2020-08-18"),(21,937,0,77,40,"2020-08-19"),(21,1839,0,20,42,"2020-08-20"),
        (21,1837,0,31,30,"2020-08-21"),(21,1639,0,32,46,"2020-08-22"),(21,827,1,23,37,"2020-08-23"),(21,1432,1,45,39,"2020-08-24"),
        (21,803,0,14,52,"2020-08-25"),(21,1329,0,82,52,"2020-08-26"),(21,1748,0,81,12,"2020-08-27"),(21,1522,1,50,50,"2020-08-28"),
        (21,1432,2,39,42,"2020-08-29"),(21,1238,1,68,51,"2020-08-30"),(21,2242,0,54,32,"2020-08-31"),
        (22,2403,0,52,43,"2020-08-01"),(22,2912,0,61,30,"2020-08-02"),(22,1098,1,83,21,"2020-08-03"),(22,2183,2,61,39,"2020-08-04"),
        (22,2421,0,19,37,"2020-08-05"),(22,2182,1,18,29,"2020-08-06"),(22,1128,0,66,55,"2020-08-07"),(22,2382,0,53,48,"2020-08-08"),
        (22,2018,0,22,39,"2020-08-09"),(22,1592,0,28,27,"2020-08-10"),(22,829,0,47,30,"2020-08-11"),(22,2110,0,20,20,"2020-08-12"),
        (22,1732,0,49,28,"2020-08-13"),(22,1037,0,68,21,"2020-08-14"),(22,1638,0,29,17,"2020-08-15"),(22,1044,0,16,44,"2020-08-16"),
        (22,1731,1,8,22,"2020-08-17"),(22,1111,0,72,19,"2020-08-18"),(22,1543,0,35,39,"2020-08-19"),(22,937,0,59,21,"2020-08-20"),
        (22,1372,0,3,53,"2020-08-21"),(22,1762,0,18,35,"2020-08-22"),(22,1472,0,13,28,"2020-08-23"),(22,2493,0,20,46,"2020-08-24"),
        (22,1138,0,13,42,"2020-08-25"),(22,1662,0,9,13,"2020-08-26"),(22,1600,0,95,8,"2020-08-27"),(22,891,0,39,53,"2020-08-28"),
        (22,1240,0,72,31,"2020-08-29"),(22,1144,0,23,38,"2020-08-30"),(22,1593,1,52,29,"2020-08-31"),
        (23,222,2,50,55,"2020-08-01"),(23,62,0,60,32,"2020-08-02"),(23,26,0,30,22,"2020-08-03"),(23,29,1,7,67,"2020-08-04"),
        (23,169,1,62,42,"2020-08-05"),(23,173,0,47,51,"2020-08-06"),(23,135,0,18,48,"2020-08-07"),(23,99,0,50,15,"2020-08-08"),
        (23,82,0,38,46,"2020-08-09"),(23,200,0,23,30,"2020-08-10"),(23,147,0,21,49,"2020-08-11"),(23,178,0,42,21,"2020-08-12"),
        (23,183,0,30,51,"2020-08-13"),(23,144,0,28,36,"2020-08-14"),(23,168,0,30,30,"2020-08-15"),(23,299,2,30,8,"2020-08-16"),
        (23,283,0,41,29,"2020-08-17"),(23,152,2,19,45,"2020-08-18"),(23,291,0,34,31,"2020-08-19"),(23,219,1,18,19,"2020-08-20"),
        (23,212,0,39,44,"2020-08-21"),(23,169,0,29,39,"2020-08-22"),(23,53,0,29,34,"2020-08-23"),(23,178,0,40,49,"2020-08-24"),
        (23,192,1,50,28,"2020-08-25"),(23,98,0,25,45,"2020-08-26"),(23,22,1,32,85,"2020-08-27"),(23,155,0,33,51,"2020-08-28"),
        (23,73,0,40,46,"2020-08-29"),(23,54,1,14,25,"2020-08-30"),(23,35,3,12,90,"2020-08-31"),
        (25,1839,4,42,19,"2020-08-01"),(25,1555,0,60,6,"2020-08-02"),(25,1412,0,21,7,"2020-08-03"),(25,1242,5,29,14,"2020-08-04"),
        (25,1537,2,32,17,"2020-08-05"),(25,1492,1,32,0,"2020-08-06"),(25,1500,1,27,29,"2020-08-07"),(25,1039,2,37,28,"2020-08-08"),
        (25,1392,0,59,8,"2020-08-09"),(25,1637,0,43,12,"2020-08-10"),(25,902,1,22,49,"2020-08-11"),(25,1183,0,81,8,"2020-08-12"),
        (25,892,3,24,13,"2020-08-13"),(25,1228,3,50,32,"2020-08-14"),(25,1182,4,38,7,"2020-08-15"),(25,1340,0,70,21,"2020-08-16"),
        (25,989,0,9,17,"2020-08-17"),(25,1447,0,72,12,"2020-08-18"),(25,1632,0,83,11,"2020-08-19"),(25,1483,3,63,18,"2020-08-20"),
        (25,1892,4,47,20,"2020-08-21"),(25,963,0,52,27,"2020-08-22"),(25,1701,0,28,17,"2020-08-23"),(25,1204,4,57,19,"2020-08-24"),
        (25,792,1,56,19,"2020-08-25"),(25,1284,0,43,12,"2020-08-26"),(25,1132,5,32,19,"2020-08-27"),(25,1874,1,48,2,"2020-08-28"),
        (25,1638,0,72,11,"2020-08-29"),(25,1195,4,50,22,"2020-08-30"),(25,1233,2,58,4,"2020-08-31"),
        (26,412,5,59,42,"2020-08-01"),(26,298,5,77,45,"2020-08-02"),(26,429,7,34,47,"2020-08-03"),(26,404,8,44,60,"2020-08-04"),
        (26,383,13,38,30,"2020-08-05"),(26,288,7,61,15,"2020-08-06"),(26,502,4,17,50,"2020-08-07"),(26,593,6,29,36,"2020-08-08"),
        (26,492,9,28,24,"2020-08-09"),(26,232,13,21,41,"2020-08-10"),(26,489,3,20,29,"2020-08-11"),(26,605,4,39,52,"2020-08-12"),
        (26,502,2,35,41,"2020-08-13"),(26,399,15,23,33,"2020-08-14"),(26,400,3,40,33,"2020-08-15"),(26,472,13,35,49,"2020-08-16"),
        (26,664,8,18,33,"2020-08-17"),(26,477,6,41,38,"2020-08-18"),(26,302,7,67,28,"2020-08-19"),(26,533,2,58,30,"2020-08-20"),
        (26,544,11,55,25,"2020-08-21"),(26,425,3,57,29,"2020-08-22"),(26,239,14,50,33,"2020-08-23"),(26,421,6,62,43,"2020-08-24"),
        (26,372,13,39,59,"2020-08-25"),(26,362,1,14,40,"2020-08-26"),(26,277,3,60,48,"2020-08-27"),(26,388,10,50,50,"2020-08-28"),
        (26,301,8,78,30,"2020-08-29"),(26,311,9,92,3,"2020-08-30"),(26,399,2,67,21,"2020-08-31"),
        (27,183,5,15,37,"2020-08-01"),(27,283,6,3,29,"2020-08-02"),(27,156,9,33,25,"2020-08-03"),(27,253,14,8,39,"2020-08-04"),
        (27,200,7,22,28,"2020-08-05"),(27,109,8,16,21,"2020-08-06"),(27,161,7,18,48,"2020-08-07"),(27,200,5,12,62,"2020-08-08"),
        (27,126,4,13,31,"2020-08-09"),(27,193,0,7,36,"2020-08-10"),(27,98,5,16,53,"2020-08-11"),(27,121,3,17,30,"2020-08-12"),
        (27,255,9,28,20,"2020-08-13"),(27,244,9,37,24,"2020-08-14"),(27,188,11,21,37,"2020-08-15"),(27,249,6,24,24,"2020-08-16"),
        (27,242,1,31,23,"2020-08-17"),(27,237,15,15,57,"2020-08-18"),(27,264,6,7,54,"2020-08-19"),(27,255,1,41,19,"2020-08-20"),
        (27,251,13,7,44,"2020-08-21"),(27,281,2,30,34,"2020-08-22"),(27,338,3,23,43,"2020-08-23"),(27,201,13,8,57,"2020-08-24"),
        (27,277,11,13,38,"2020-08-25"),(27,299,12,28,19,"2020-08-26"),(27,139,1,22,47,"2020-08-27"),(27,267,10,16,40,"2020-08-28"),
        (27,178,8,19,27,"2020-08-29"),(27,177,17,9,45,"2020-08-30"),(27,87,12,11,18,"2020-08-31"),
        (28,1124,14,5,12,"2020-08-01"),(28,1044,7,7,7,"2020-08-02"),(28,755,13,6,6,"2020-08-03"),(28,1533,18,7,2,"2020-08-04"),
        (28,679,18,8,4,"2020-08-05"),(28,677,11,1,8,"2020-08-06"),(28,632,16,7,0,"2020-08-07"),(28,1034,7,3,3,"2020-08-08"),
        (28,575,2,14,4,"2020-08-09"),(28,732,15,25,0,"2020-08-10"),(28,687,2,3,15,"2020-08-11"),(28,543,9,11,16,"2020-08-12"),
        (28,533,7,4,9,"2020-08-13"),(28,789,4,19,17,"2020-08-14"),(28,555,1,14,9,"2020-08-15"),(28,566,12,13,27,"2020-08-16"),
        (28,423,14,3,5,"2020-08-17"),(28,643,9,7,2,"2020-08-18"),(28,502,7,5,5,"2020-08-19"),(28,624,13,1,17,"2020-08-20"),
        (28,1463,5,0,11,"2020-08-21"),(28,611,12,8,11,"2020-08-22"),(28,487,14,3,16,"2020-08-23"),(28,620,8,22,0,"2020-08-24"),
        (28,699,11,14,8,"2020-08-25"),(28,586,10,17,0,"2020-08-26"),(28,679,9,0,4,"2020-08-27"),(28,1111,5,9,6,"2020-08-28"),
        (28,581,8,21,10,"2020-08-29"),(28,581,6,4,13,"2020-08-30"),(28,700,6,7,13,"2020-08-31"),
        (29,1524,0,56,25,"2020-08-01"),(29,987,0,42,39,"2020-08-02"),(29,1439,1,34,38,"2020-08-03"),(29,1188,0,41,37,"2020-08-04"),
        (29,1730,0,48,39,"2020-08-05"),(29,1771,0,49,43,"2020-08-06"),(29,1378,0,41,29,"2020-08-07"),(29,1263,0,33,59,"2020-08-08"),
        (29,1930,1,27,57,"2020-08-09"),(29,1849,1,38,36,"2020-08-10"),(29,1598,0,39,37,"2020-08-11"),(29,1730,0,21,68,"2020-08-12"),
        (29,1847,0,64,19,"2020-08-13"),(29,1667,0,30,68,"2020-08-14"),(29,1702,0,68,31,"2020-08-15"),(29,2740,0,28,46,"2020-08-16"),
        (29,1222,0,30,34,"2020-08-17"),(29,1493,4,27,59,"2020-08-18"),(29,1832,2,84,18,"2020-08-19"),(29,2489,0,34,32,"2020-08-20"),
        (29,1352,2,74,40,"2020-08-21"),(29,1339,0,18,22,"2020-08-22"),(29,1440,0,36,36,"2020-08-23"),(29,2009,2,50,56,"2020-08-24"),
        (29,1667,0,55,36,"2020-08-25"),(29,1028,0,37,65,"2020-08-26"),(29,927,0,25,39,"2020-08-27"),(29,2021,0,33,13,"2020-08-28"),
        (29,1599,0,61,38,"2020-08-29"),(29,1088,0,70,39,"2020-08-30"),(29,1073,1,47,27,"2020-08-31"),
        (30,329,2,20,45,"2020-08-01"),(30,222,8,10,47,"2020-08-02"),(30,173,12,4,41,"2020-08-03"),(30,177,13,7,52,"2020-08-04"),
        (30,710,6,10,42,"2020-08-05"),(30,334,2,2,37,"2020-08-06"),(30,123,16,8,39,"2020-08-07"),(30,189,15,8,46,"2020-08-08"),
        (30,400,5,7,29,"2020-08-09"),(30,354,6,20,78,"2020-08-10"),(30,394,9,6,79,"2020-08-11"),(30,207,9,11,83,"2020-08-12"),
        (30,329,0,4,47,"2020-08-13"),(30,378,6,4,44,"2020-08-14"),(30,338,14,7,39,"2020-08-15"),(30,284,2,6,93,"2020-08-16"),
        (30,284,0,6,36,"2020-08-17"),(30,312,13,1,39,"2020-08-18"),(30,267,17,6,29,"2020-08-19"),(30,351,8,15,43,"2020-08-20"),
        (30,244,4,0,67,"2020-08-21"),(30,200,4,14,29,"2020-08-22"),(30,313,11,12,58,"2020-08-23"),(30,273,14,14,55,"2020-08-24"),
        (30,285,12,11,39,"2020-08-25"),(30,173,5,9,55,"2020-08-26"),(30,461,10,13,64,"2020-08-27"),(30,290,17,29,61,"2020-08-28"),
        (30,197,7,8,51,"2020-08-29"),(30,183,15,4,64,"2020-08-30"),(30,245,16,50,43,"2020-08-31")]

df_consumos_diarios = spark.createDataFrame(data, columns)
df_consumos_diarios.write.mode("overwrite").option("encoding", "UTF-8").saveAsTable('df_consumos_diarios')



In [0]:
display(df_consumos_diarios)

id_cliente,consumo_datos_MB,sms_enviados,minutos_llamadas_movil,minutos_llamadas_fijo,fecha
1,664,3,25,0,2020-08-01
1,943,0,12,5,2020-08-02
1,1035,1,46,10,2020-08-03
1,760,3,17,0,2020-08-04
1,1409,1,31,4,2020-08-05
1,1627,0,20,0,2020-08-06
1,739,3,21,0,2020-08-07
1,760,0,12,0,2020-08-08
1,928,0,0,24,2020-08-09
1,829,2,46,0,2020-08-10


**Caso 1. Mostrar por pantalla el número TOTAL de clientes que tenían en el mes anterior más de un contrato con la compañía**

In [0]:
#se debe iniciar llamando el DF correspondiente
df_facturas_mes_ant = spark.read.table("df_facturas_mes_ant")
df_facturas_mes_ant.printSchema()

root
 |-- id_cliente: long (nullable = true)
 |-- id_oferta: long (nullable = true)
 |-- importe: double (nullable = true)
 |-- fecha: string (nullable = true)



In [0]:
df_facturas_mes_ant = spark.read.table("df_facturas_mes_ant")  #se muestra el DF de clientes del mes anterior
display(df_facturas_mes_ant)

id_cliente,id_oferta,importe,fecha
23,3,85.0,2020-07-31
24,3,85.0,2020-07-31
25,9,42.5,2020-07-31
25,10,31.99,2020-07-31
26,4,95.0,2020-07-31
27,8,84.0,2020-07-31
27,9,41.5,2020-07-31
29,6,124.5,2020-07-31
30,1,70.0,2020-07-31
30,2,70.0,2020-07-31


In [0]:
df_clientes = spark.read.table("df_clientes")
df_clientes = df_clientes.select("id_cliente","nombre") 
display(df_clientes)

id_cliente,nombre
10,Pablo Lopez
11,Luis Rodrigues
12,Alvaro Monroy
13,Victor Ruiz
14,Melisa Aguado
15,Cristian Cuadrado
25,Ines Barcero
26,Celia Castro
27,Roberta Varado
28,Walter Ramos


In [0]:
# se procede a dar respuesta a contar o indicar cuantos clientes tenian mas de un contrato 
# Primero debemos proceder a agrupar por cliente y contar los contratos
conteo_de_clientes = df_facturas_mes_ant.groupBy("id_cliente").agg(count("id_cliente").alias("conteo_contratos"))

# Se procede a crear un iltro clientes con más de un contrato
clientes_con_varios_contratos = conteo_de_clientes.filter(col("conteo_contratos") > 1)

# Con esto se puede ver el número total de estos clientes
clientes_con_varios_contratos.count()

# Finalmente se muestra el resultado
print("El número TOTAL de Clientes que tenian el mes anterior mas de un contrato corresponde a un total de: " + str(clientes_con_varios_contratos.count())) 

El número TOTAL de Clientes que tenian el mes anterior mas de un contrato corresponde a un total de: 8


**Estos son los clientes que tienen mas de un contrato**  Esto no es solicitado, pero lo aplico como un metodo de verificacion, que en total tenemos 8 clientes que presentan mas de un contrato

In [0]:

from pyspark.sql.window import Window
window_cliente = Window.partitionBy("id_cliente").orderBy(desc("importe"))

df_facturas_mes_ant = df_facturas_mes_ant.join(df_clientes,["id_cliente"],"left")
df_facturas_mes_ant = df_facturas_mes_ant.withColumn("count_contratos", row_number().over(window_cliente))
# Como puede haber ventanas con más de 2 elementos (por ejemplo el caso del id_cliente 1), creamos otra columna ("total_contratos") que nos informe del número maximo de la columna que acabamos de crear ("count_contratos").
df_facturas_mes_ant = df_facturas_mes_ant.withColumn("total_contratos", max("count_contratos").over(window_cliente.rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)))
df_facturas_mes_ant = df_facturas_mes_ant.filter(col("total_contratos") > 1)
df_facturas_mes_ant = df_facturas_mes_ant.dropDuplicates(["id_cliente"])
df_facturas_mes_ant = df_facturas_mes_ant.drop("id_oferta","importe","fecha","count_contratos")
window_indice = Window.orderBy(sql_functions.monotonically_increasing_id())
df_facturas_mes_ant = df_facturas_mes_ant.withColumn("indice", sql_functions.row_number().over(window_indice))
df_facturas_mes_ant.show(n=40)

+----------+-----------------+---------------+------+
|id_cliente|           nombre|total_contratos|indice|
+----------+-----------------+---------------+------+
|         1|      Pablo Perez|              3|     1|
|         5|       Ana Robles|              2|     2|
|        15|Cristian Cuadrado|              2|     3|
|        18|       Laura Luiz|              2|     4|
|        22|     Clara Suarez|              2|     5|
|        25|     Ines Barcero|              2|     6|
|        27|   Roberta Varado|              2|     7|
|        30|    Marta Rodrigo|              2|     8|
+----------+-----------------+---------------+------+



**Caso 2. Generar un nuevo dataframe de facturas del mes actual que asigne un 7% de descuento a todos los contratos de clientes que ya existían en el mes anterior (los contratos de los clientes nuevos seguirán con el mismo importe). Mantener la columna "importe" y crear una nueva columna "importe_dto" con el nuevo importe, casteada a 2 decimales. Mostrar además el resultado ordenado por los campos id_cliente ascendente, importe descendente.**

In [0]:
#llamar los dataframes necesarios 
df_facturas_mes_ant = spark.read.table("df_facturas_mes_ant")
df_facturas_mes_actual = spark.read.table("df_facturas_mes_actual")
display(df_facturas_mes_ant)
display(df_facturas_mes_actual)

id_cliente,id_oferta,importe,fecha
23,3,85.0,2020-07-31
24,3,85.0,2020-07-31
25,9,42.5,2020-07-31
25,10,31.99,2020-07-31
26,4,95.0,2020-07-31
27,8,84.0,2020-07-31
27,9,41.5,2020-07-31
29,6,124.5,2020-07-31
30,1,70.0,2020-07-31
30,2,70.0,2020-07-31


id_cliente,id_oferta,importe,fecha
25,10,29.7,2020-08-31
26,4,95.0,2020-08-31
27,8,84.0,2020-08-31
28,5,102.5,2020-08-31
29,6,121.5,2020-08-31
30,1,70.0,2020-08-31
30,2,71.0,2020-08-31
1,6,124.5,2020-08-31
1,6,118.6,2020-08-31
2,4,95.0,2020-08-31


In [0]:
 # Procedemos a obtener los clientes que tenían contratos el mes anterior
clientes_mes_anterior = df_facturas_mes_ant.select("id_cliente").distinct()

# Con el siguiente comando se procede a unir los DataFrames del mes actual con los clientes del mes anterior
df_facturas_con_descuento = df_facturas_mes_actual.join(clientes_mes_anterior, ["id_cliente"],"left")

# Segun lo solicitado se requiere que se le aplique un  7% de descuento a los clientes que existían el mes anterior (tipo de beneficio a la fidelidad de clientes)
df_facturas_con_descuento = df_facturas_con_descuento.withColumn("importe_dto",when(col("id_cliente").isNotNull(), round(col("importe") * 0.93, 2)).otherwise(col("importe")))

# Ordenar por id_cliente ascendente y importe descendente
df_facturas_con_descuento_ordenado = df_facturas_con_descuento.orderBy(col("id_cliente").asc(), col("importe").desc())

# Se procede a mostrar el resultado
display(df_facturas_con_descuento_ordenado)

id_cliente,id_oferta,importe,fecha,importe_dto
1,6,124.5,2020-08-31,115.79
1,6,118.6,2020-08-31,110.3
2,4,95.0,2020-08-31,88.35
3,5,102.5,2020-08-31,95.33
4,1,70.0,2020-08-31,65.1
5,7,80.5,2020-08-31,74.87
5,9,40.5,2020-08-31,37.67
6,8,84.0,2020-08-31,78.12
7,4,95.0,2020-08-31,88.35
8,5,100.5,2020-08-31,93.47


**Caso 3. Por problemas de saturación en la red debido al incremento en el uso de datos por el confinamiento debido a la COVID-19, se decide limitar el uso de datos este mes subiendo la tarifa de todas las ofertas que incluyan "datos ilimitados" en un 15%. Obtener un DF con las mismas columnas que el DF de facturas del mes actual y a mayores la columna "importe_dto"**.

In [0]:
#llamar los dataframes necesarios 
df_ofertas = spark.read.table("df_ofertas")
df_facturas_mes_actual = spark.read.table("df_facturas_mes_actual")
display(df_ofertas)
display(df_facturas_mes_actual)

id_oferta,descripcion,importe
6,Fibra Optica 600MB + Fijo ilimitado + Movil llamadas y datos ilimitados + Television,124.5
3,Fibra Optica 300MB + Movil llamadas y datos ilimitados + Television,85.0
7,Fibra Optica 300MB + Movil llamadas ilimitadas + Television,80.5
8,Fibra Optica 300MB + Movil datos ilimitados + Television,84.0
1,Fibra Optica 300MB + Fijo ilimitado + Television,70.0
2,Fibra Optica 600MB + Fijo ilimitado + Television,78.5
4,Fibra Optica 600MB + Movil llamadas y datos ilimitados + Television,95.0
5,Fibra Optica 300MB + Fijo ilimitado + Movil llamadas y datos ilimitados,102.5
9,Fijo llamadas ilimitadas + Movil llamadas y datos ilimitados,42.5
10,Fijo llamadas ilimitadas + Movil llamadas ilimitadas,31.99


id_cliente,id_oferta,importe,fecha
25,10,29.7,2020-08-31
26,4,95.0,2020-08-31
27,8,84.0,2020-08-31
28,5,102.5,2020-08-31
29,6,121.5,2020-08-31
30,1,70.0,2020-08-31
30,2,71.0,2020-08-31
1,6,124.5,2020-08-31
1,6,118.6,2020-08-31
2,4,95.0,2020-08-31


In [0]:
# Segun el enunciado se debe proceder con un aumento del 15% a las ofertas que incluyan "datos ilimitados"
df_facturas_ajustadas = df_facturas_mes_actual.alias("actual").join(df_ofertas.withColumnRenamed("importe", "importe_oferta"), "id_oferta", "left"). \
                                    withColumn("importe_dto", when(lower(col("descripcion")).contains("datos ilimitados"), col("importe_oferta") * 1.15).otherwise(col("importe_oferta"))). \
                                    select("id_cliente", "id_oferta", "importe", "fecha", "importe_dto")
 
display(df_facturas_ajustadas)

id_cliente,id_oferta,importe,fecha,importe_dto
25,10,29.7,2020-08-31,31.99
26,4,95.0,2020-08-31,109.24999999999999
27,8,84.0,2020-08-31,96.6
28,5,102.5,2020-08-31,117.87499999999999
29,6,121.5,2020-08-31,143.17499999999998
30,1,70.0,2020-08-31,70.0
30,2,71.0,2020-08-31,78.5
1,6,124.5,2020-08-31,143.17499999999998
1,6,118.6,2020-08-31,143.17499999999998
2,4,95.0,2020-08-31,109.24999999999999


**Vamos a mostrar los mismos resultados pero con solo dos decimales**

In [0]:
df_facturas_ajustadas = df_facturas_mes_actual.alias("actual").join(df_ofertas.withColumnRenamed("importe", "importe_oferta"), "id_oferta", "left"). \
                                    withColumn("importe_dto", round(when(lower(col("descripcion")).contains("datos ilimitados"), col("importe_oferta") * 1.15).otherwise(col("importe_oferta")),2))\
                                    .select("id_cliente", "id_oferta", "importe", "fecha", "importe_dto")
display(df_facturas_ajustadas)

id_cliente,id_oferta,importe,fecha,importe_dto
25,10,29.7,2020-08-31,31.99
26,4,95.0,2020-08-31,109.25
27,8,84.0,2020-08-31,96.6
28,5,102.5,2020-08-31,117.87
29,6,121.5,2020-08-31,143.17
30,1,70.0,2020-08-31,70.0
30,2,71.0,2020-08-31,78.5
1,6,124.5,2020-08-31,143.17
1,6,118.6,2020-08-31,143.17
2,4,95.0,2020-08-31,109.25


**4. Crear una nueva variable "grupo_edad" que agrupe a los clientes, tanto del mes_actual como del mes_anterior en 4 rangos según su edad asignando valores del 1 - 4 segun el rango al que pertenezcan (18-25 (1), 26-40 (2), 41-65 (3), >65 (4)). Obtener una tabla resumen que extraiga para cada para cada uno de los 4 grupos identificados la MEDIA de consumo de datos, sms enviados, minutos_movil, minutos_fijo, con todos los campos casteados a 2 decimales y ordenar el DF por grupo_edad ascendente. Extraer conclusiones.**

In [0]:
spark.catalog.listTables() # este codigo nos permite ver los DF o talbes disponibles (creados hasta el momento)

Out[121]: [Table(name='df_clientes', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='df_consumos_diarios', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='df_facturas_mes_actual', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='df_facturas_mes_ant', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='df_ofertas', database='default', description=None, tableType='MANAGED', isTemporary=False)]

In [0]:
df_clientes = spark.read.table("df_clientes") #leemos o llamamos el Df_clientes de su tabla original
display(df_clientes)

id_cliente,nombre,edad,sexo,provincia,pais
10,Pablo Lopez,40,M,Sevilla,España
11,Luis Rodrigues,20,M,Bogota,Colombia
12,Alvaro Monroy,75,M,Monterrey,Mexico
13,Victor Ruiz,53,M,Valladolid,España
14,Melisa Aguado,30,F,Bogota,Colombia
15,Cristian Cuadrado,52,M,Guadalajara,Mexico
25,Ines Barcero,29,F,Guadalajara,Mexico
26,Celia Castro,47,F,Barcelona,España
27,Roberta Varado,64,F,Cali,Colombia
28,Walter Ramos,54,M,Cali,Colombia


In [0]:
df_facturas_mes_ant = spark.read.table("df_facturas_mes_ant")   #asi como tambien se procede a enunciar / llamar los DF de mes anterior como el del mes actual
df_facturas_mes_actual =  spark.read.table("df_facturas_mes_actual")

**En este caso vamos a agregar el rango de las edades em el DF de clientes**


In [0]:
#para efectos de este ejercicio la informacion de rangos se le va agregar directamente al Df Clientes, esto porque es el Df que mantiene la informacion general de los clientes
# y en caso de requerir estos datos en otros DF solo basta con realizar un JOIN  con esta tabla y se tendra disponible.
df_clientes= df_clientes.withColumn("grupo_edad", when(col("edad") < 26, lit(1))
                                         .when(col("edad").between(26,40),lit(2))
                                         .when(col("edad").between(41,65),lit(3))
                                         .otherwise(lit(4)))
display(df_clientes)

id_cliente,nombre,edad,sexo,provincia,pais,grupo_edad
10,Pablo Lopez,40,M,Sevilla,España,2
11,Luis Rodrigues,20,M,Bogota,Colombia,1
12,Alvaro Monroy,75,M,Monterrey,Mexico,4
13,Victor Ruiz,53,M,Valladolid,España,3
14,Melisa Aguado,30,F,Bogota,Colombia,2
15,Cristian Cuadrado,52,M,Guadalajara,Mexico,3
25,Ines Barcero,29,F,Guadalajara,Mexico,2
26,Celia Castro,47,F,Barcelona,España,3
27,Roberta Varado,64,F,Cali,Colombia,3
28,Walter Ramos,54,M,Cali,Colombia,3


In [0]:
df_clientes_join = df_clientes.select("id_cliente","edad","grupo_edad")
df_facturas_mes_ant = df_facturas_mes_ant.join(df_clientes_join,["id_cliente"],"left").orderBy("id_cliente")
df_facturas_mes_actual = df_facturas_mes_actual.join(df_clientes_join,["id_cliente"],"left").orderBy("id_cliente")

display(df_facturas_mes_ant)

id_cliente,id_oferta,importe,fecha,edad,grupo_edad
1,6,124.5,2020-07-31,26,2
1,9,36.5,2020-07-31,26,2
1,6,118.6,2020-07-31,26,2
2,4,95.0,2020-07-31,58,3
3,5,102.5,2020-07-31,68,4
5,7,80.5,2020-07-31,41,3
5,9,40.5,2020-07-31,41,3
6,8,84.0,2020-07-31,74,4
7,4,95.0,2020-07-31,19,1
8,5,102.5,2020-07-31,49,3


In [0]:
display(df_facturas_mes_actual)

id_cliente,id_oferta,importe,fecha,edad,grupo_edad
1,6,124.5,2020-08-31,26,2
1,6,118.6,2020-08-31,26,2
2,4,95.0,2020-08-31,58,3
3,5,102.5,2020-08-31,68,4
4,1,70.0,2020-08-31,32,2
5,7,80.5,2020-08-31,41,3
5,9,40.5,2020-08-31,41,3
6,8,84.0,2020-08-31,74,4
7,4,95.0,2020-08-31,19,1
8,5,100.5,2020-08-31,49,3


In [0]:
df_consumos_diarios = spark.read.table("df_consumos_diarios")
display(df_consumos_diarios)

id_cliente,consumo_datos_MB,sms_enviados,minutos_llamadas_movil,minutos_llamadas_fijo,fecha
27,249,6,24,24,2020-08-16
27,242,1,31,23,2020-08-17
27,237,15,15,57,2020-08-18
27,264,6,7,54,2020-08-19
27,255,1,41,19,2020-08-20
27,251,13,7,44,2020-08-21
27,281,2,30,34,2020-08-22
27,338,3,23,43,2020-08-23
27,201,13,8,57,2020-08-24
27,277,11,13,38,2020-08-25


In [0]:
df_consumos_diarios_join = df_consumos_diarios.join(df_clientes_join,["id_cliente"],"left").orderBy("id_cliente")

display(df_consumos_diarios_join)

id_cliente,consumo_datos_MB,sms_enviados,minutos_llamadas_movil,minutos_llamadas_fijo,fecha,edad,grupo_edad
1,739,3,21,0,2020-08-07,26,2
1,603,0,41,0,2020-08-13,26,2
1,819,1,95,18,2020-08-27,26,2
1,760,0,60,12,2020-08-28,26,2
1,732,0,34,102,2020-08-14,26,2
1,302,0,29,0,2020-08-29,26,2
1,497,3,14,0,2020-08-30,26,2
1,729,1,0,50,2020-08-15,26,2
1,1281,2,21,0,2020-08-31,26,2
1,760,1,31,0,2020-08-16,26,2


**Para cada uno de los 4 grupos identificados la MEDIA de consumo de datos, sms enviados, minutos_movil, minutos_fijo, con todos los campos casteados a 2 decimales y ordenar el DF por grupo_edad ascendente. Extraer conclusiones.**

In [0]:
# Calcular la media de consumo de datos, sms enviados, minutos_movil y minutos_fijo por grupo_edad
#  vamos que ahora toca generar no solo los valores sino tambien una tabla resumen, lo cual se procede a usar un redondeo a 2 digitos 
df_consumos_diarios_resumen = df_consumos_diarios_join.groupBy("grupo_edad").agg(
      round(mean("consumo_datos_MB"), 2).alias("media_consumo_datos"),
    round(mean("sms_enviados"), 2).alias("media_sms_enviados"),
    round(mean("minutos_llamadas_movil"), 2).alias("media_minutos_movil"),
    round(mean("minutos_llamadas_fijo"), 2).alias("media_minutos_fijo")
).orderBy("grupo_edad")

# Mostrar el resultado
display(df_consumos_diarios_resumen)

grupo_edad,media_consumo_datos,media_sms_enviados,media_minutos_movil,media_minutos_fijo
1,1338.24,0.54,18.97,10.86
2,1082.89,1.13,34.88,21.76
3,473.29,5.25,32.44,26.43
4,171.19,6.16,18.09,35.75


### Conclusiones sobre el resultado del resumen anterior

rangos según su edad asignando valores del 1 - 4 segun el rango al que pertenezcan (18-25 (1), 26-40 (2), 41-65 (3), >65 (4))

* **Media Consumo Datos**  : **El grupo de edad - 1  18 a 25**, presenta la media de consumo de datos mas alta, y la misma disminuye a mayor rango de edad
* **Media Mensajes Enviados SMS** : **El grupo numero 4** mayor a 65. Es que presenta un mayor uso de los mensajes de texto con una media 6.
* **Media de Minutos Movil** : Los grupos de eda 2 y 3 son los que mas consumen los mintiso movil presentando una media de 34.88 los del grupo 2, y una media 32.44 los del grupo 3.
* **Media de Minutos Fijo** : Aquellos que pertenecen a los grupo de edad 4. presentan el mayor consumo de minutos fijo

Este breve ejercicio permite conocer con una breve exploracion de los datos, el comportamiento de consumo en telefonia por las personas segmentadas por la edad, a lo que se le puede dar seguimiento y la empresa podria generar
promociones segun los ragos de preferencia de la segmentacion de los clientes por edad.




**5. Se quiere realizar un estudio por Sexo para analizar si son las mujeres o los hombres quiénes consumen más datos durante el fin de semana y hacen más llamadas desde el móvil. Para ello se deberá, sin ayuda de un calendario, extraer el día de la semana al que corresponde cada una de las fechas del mes de
Agosto para saber cuáles son fin de semana (se consideran días de fin de semana el viernes, sábado y domingo). </br> El DF a obtener deberá tener 2 registros con las columnas: sexo, total_mins_movil_finde, total_datos_moviles_finde. </br> Extraer conclusiones tras presentar el DF resultante.**

In [0]:
spark.catalog.listTables() # este codigo nos permite ver los DF o talbes disponibles (creados hasta el momento)

Out[130]: [Table(name='df_clientes', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='df_consumos_diarios', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='df_facturas_mes_actual', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='df_facturas_mes_ant', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='df_ofertas', database='default', description=None, tableType='MANAGED', isTemporary=False)]

In [0]:
df_consumos_diarios = spark.read.table("df_consumos_diarios")
df_clientes = spark.read.table("df_clientes")
df_clientes_join= df_clientes.select("id_cliente","sexo")

In [0]:
df_consumos_diarios_join = df_consumos_diarios.join(df_clientes_join,["id_cliente"],"left")

In [0]:
display(df_consumos_diarios_join)

id_cliente,consumo_datos_MB,sms_enviados,minutos_llamadas_movil,minutos_llamadas_fijo,fecha,sexo
27,249,6,24,24,2020-08-16,F
27,242,1,31,23,2020-08-17,F
27,237,15,15,57,2020-08-18,F
27,264,6,7,54,2020-08-19,F
27,255,1,41,19,2020-08-20,F
27,251,13,7,44,2020-08-21,F
27,281,2,30,34,2020-08-22,F
27,338,3,23,43,2020-08-23,F
27,201,13,8,57,2020-08-24,F
27,277,11,13,38,2020-08-25,F


In [0]:
df_consumos_diarios_join = df_consumos_diarios_join.withColumn('fecha', col('fecha').cast('date'))
df_consumos_diarios_join = df_consumos_diarios_join.withColumn("dia_semana", dayofweek(col("fecha")))


In [0]:
display(df_consumos_diarios_join)

id_cliente,consumo_datos_MB,sms_enviados,minutos_llamadas_movil,minutos_llamadas_fijo,fecha,sexo,dia_semana
27,249,6,24,24,2020-08-16,F,1
27,242,1,31,23,2020-08-17,F,2
27,237,15,15,57,2020-08-18,F,3
27,264,6,7,54,2020-08-19,F,4
27,255,1,41,19,2020-08-20,F,5
27,251,13,7,44,2020-08-21,F,6
27,281,2,30,34,2020-08-22,F,7
27,338,3,23,43,2020-08-23,F,1
27,201,13,8,57,2020-08-24,F,2
27,277,11,13,38,2020-08-25,F,3


In [0]:
df_consumos_diarios_join_finde = df_consumos_diarios_join.filter(col('dia_semana').isin([5, 6, 7]))

# Agrupar por sexo y sumar minutos de móvil y datos móviles
df_consumos_diarios_join_finde = df_consumos_diarios_join_finde.groupBy('sexo').agg(spark_sum('minutos_llamadas_movil').alias('total_mins_movil_finde'),
spark_sum('consumo_datos_MB').alias('total_datos_moviles_finde'))


*El DF a obtener deberá tener 2 registros con las columnas: sexo, total_mins_movil_finde, total_datos_moviles_finde.*

In [0]:
display(df_consumos_diarios_join_finde)

sexo,total_mins_movil_finde,total_datos_moviles_finde
F,6732,137824
M,3303,127323


### Conclusiones de los resultados  

Segun el cuadro resumen anterior se denotan algunas diferencias claras :

* **Total de minutos moviles**  Se denota un mayor consumo por parte de las mujeres en el total de llamadas o minutos movil, Mujeres un total de 6732 minutos contra los Hombres 3303, praticamente un consumo del doble

* **Total de Datos Moviles**  Existe un consumo similar entre ambos, siendo en este caso nuevamente las mujeres las que consumen mas los datos los fines de semana


Estos resultados nos permite determinar que durante los fines de semana los clientes tendran mayor consumo de datos y llamadas , y las muejeres van a demandar un mayor cantidad de llamadas, la segmentacion y promociones de los clientes se puede hacer tomando en cuenta esta informacion.


**6. Obtener un DF que contenga 4 registros, que serán el cliente de cada grupo edad que más datos móviles ha consumido durante los 15 primeros días del mes de Agosto (día 15 incluido en el cálculo). El DF deberá contener las columnas: nombre, edad, grupo_edad, datos_moviles_total_15, max_sms_enviados_15** </br> (max_sms_enviados_15 contiene el máximo de sms enviados en un día por el cliente con más datos_moviles consumidos de cada grupo_edad durante esos 15 primeros días del mes). </br>
**Extraer conclusiones en cuanto a datos consumidos y sms enviados por cada grupo_edad.**

In [0]:
spark.catalog.listTables() # este codigo nos permite ver los DF o talbes disponibles (creados hasta el momento)

Out[138]: [Table(name='df_clientes', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='df_consumos_diarios', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='df_facturas_mes_actual', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='df_facturas_mes_ant', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='df_ofertas', database='default', description=None, tableType='MANAGED', isTemporary=False)]

In [0]:
# Se procede a llamar los dataframes necesarios, si se requiere agun otro, entonces se procedera agregar y realizar las transformaciones necesarias
df_consumos_diarios = spark.read.table("df_consumos_diarios")
df_clientes = spark.read.table("df_clientes")

In [0]:
display(df_clientes)

id_cliente,nombre,edad,sexo,provincia,pais
10,Pablo Lopez,40,M,Sevilla,España
11,Luis Rodrigues,20,M,Bogota,Colombia
12,Alvaro Monroy,75,M,Monterrey,Mexico
13,Victor Ruiz,53,M,Valladolid,España
14,Melisa Aguado,30,F,Bogota,Colombia
15,Cristian Cuadrado,52,M,Guadalajara,Mexico
25,Ines Barcero,29,F,Guadalajara,Mexico
26,Celia Castro,47,F,Barcelona,España
27,Roberta Varado,64,F,Cali,Colombia
28,Walter Ramos,54,M,Cali,Colombia


In [0]:
df_consumos_diarios.show()

+----------+----------------+------------+----------------------+---------------------+----------+
|id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo|     fecha|
+----------+----------------+------------+----------------------+---------------------+----------+
|         1|             664|           3|                    25|                    0|2020-08-01|
|         1|             943|           0|                    12|                    5|2020-08-02|
|         1|            1035|           1|                    46|                   10|2020-08-03|
|         1|             760|           3|                    17|                    0|2020-08-04|
|         1|            1409|           1|                    31|                    4|2020-08-05|
|         1|            1627|           0|                    20|                    0|2020-08-06|
|         1|             739|           3|                    21|                    0|2020-08-07|
|         

In [0]:
# Obtener un DF que contenga solo a los clientes nuevos de este mes con cuatro columnas:
# “nombre_cliente_nuevo”, edad, “importe_total_mes_actual”, “total_minutos”.nombre, edad, grupo_edad
# este codigo permite realizar la seleccion de los campos necesarios del DF clientes que se requeire para 
#cumplir con lo que nos estan solicitando
df_clientes_join= df_clientes.select("id_cliente","nombre", "edad")
df_clientes_join= df_clientes_join.withColumn("grupo_edad", when(col("edad") < 26, lit(1))
                                         .when(col("edad").between(26,40),lit(2))
                                         .when(col("edad").between(41,65),lit(3))
                                         .otherwise(lit(4)))
display(df_clientes_join)


id_cliente,nombre,edad,grupo_edad
10,Pablo Lopez,40,2
11,Luis Rodrigues,20,1
12,Alvaro Monroy,75,4
13,Victor Ruiz,53,3
14,Melisa Aguado,30,2
15,Cristian Cuadrado,52,3
25,Ines Barcero,29,2
26,Celia Castro,47,3
27,Roberta Varado,64,3
28,Walter Ramos,54,3


In [0]:
# esta transformacion permite que el DF clientes con los campos necesarios este con joins al DF de consumos diarios, con el campo
# de fecha necesario para realizar los analisis que se solicitan, con este Join le agregamos los campos de edad, grupo edad, nombre cliente
df_consumos_diarios_join = df_consumos_diarios.join(df_clientes_join,["id_cliente"],"left")
df_consumos_diarios_join = df_consumos_diarios_join.withColumn('fecha', col('fecha').cast('date'))
df_consumos_diarios_join = df_consumos_diarios_join.withColumn("dia_semana", dayofweek(col("fecha")))
df_consumos_diarios_join.show()

+----------+----------------+------------+----------------------+---------------------+----------+--------------+----+----------+----------+
|id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo|     fecha|        nombre|edad|grupo_edad|dia_semana|
+----------+----------------+------------+----------------------+---------------------+----------+--------------+----+----------+----------+
|        27|             249|           6|                    24|                   24|2020-08-16|Roberta Varado|  64|         3|         1|
|        27|             242|           1|                    31|                   23|2020-08-17|Roberta Varado|  64|         3|         2|
|        27|             237|          15|                    15|                   57|2020-08-18|Roberta Varado|  64|         3|         3|
|        27|             264|           6|                     7|                   54|2020-08-19|Roberta Varado|  64|         3|         4|
|        27| 

In [0]:
# Filtrar primeros 15 días del mes
df_primeros_15 = df_consumos_diarios_join.filter(dayofmonth(col("fecha")) <= 15)

# Calcular el total de datos móviles y máximo de SMS por día por cliente
df_primeros_15_agg = df_primeros_15.groupBy("id_cliente", "grupo_edad").agg(
    sum("consumo_datos_MB").alias("datos_moviles_total_15"),
    max("sms_enviados").alias("max_sms_enviados_15")
)

# Obtener el cliente con más datos móviles por grupo de edad
window_edad = Window.partitionBy("grupo_edad").orderBy(col("datos_moviles_total_15").desc())
df_top_clientes = df_primeros_15_agg.withColumn("rank", row_number().over(window_edad)).filter(col("rank") == 1).drop("rank")

# Unir con información del cliente
df_top_clientes = df_top_clientes.join(df_clientes, "id_cliente", "left").select("nombre", "edad", "grupo_edad", "datos_moviles_total_15", "max_sms_enviados_15")

display(df_top_clientes)

nombre,edad,grupo_edad,datos_moviles_total_15,max_sms_enviados_15
Clara Suarez,21,1,27665,2
Romina Verde,29,2,23603,1
Walter Ramos,54,3,11892,18
Marta Rodrigo,67,4,4657,16


### Conclusines

El cleinte que nos dice la tabla resumen con mayor conusmo de datos moviles corresponde a **Carlos** de 21 años 

Los clientes que tienen un mayor consumo, comportamiento en general, que se pueden asignar como VIP's son importantes para cualquier empresa, ademas las empresas deben buscar fidelizacion de los clientes para que auqellos clientes A,B o C, se mantengan o aumenten de categoria

Esto permite a las empresas garantizar un monto promedio de facturacion de servicios, disminucion de costos en atencion de quejas.



**7. Nos interesa averiguar los minutos de llamadas de los clientes que son nuevos este mes para realizar un estudio del impacto que tendrán en caso de producirse un pico de volumen de llamadas en la red. </br> Obtener un DF que contenga sólo a los clientes nuevos de este mes con 4 columnas: nombre_cliente_nuevo, edad, importe_total_mes_actual, total_minutos. </br> Se ordenará el DF resultante por la columna "total_minutos" que contiene el total de minutos de llamadas fijas y móviles durante este mes para cada cliente, ordenándolos de manera que el primer cliente nuevo que aparezca sea el que menos minutos de llamadas ha realizado en el mes de Agosto.</br>** NOTAS: Si hay solo 2 clientes nuevos, el DF a obtener tendrá sólo 2 registros. </br> Tener presente que a la hora de hacer un join, la clave de unión debe ser única en la tabla de la derecha o de otra manera se multiplicarán los registros resultantes de manera descontrolada.
</br> Tener en cuenta que si un cliente nuevo tiene 2 ofertas contratadas, la columna importe_mes_actual deberá contener la suma de los importes de ambas ofertas.

In [0]:
# Se procede a llamar los dataframes necesarios, si se requiere agun otro, entonces se procedera agregar y realizar las transformaciones necesarias
df_consumos_diarios = spark.read.table("df_consumos_diarios")
df_clientes = spark.read.table("df_clientes")
df_consumos_diarios = spark.read.table("df_consumos_diarios")

In [0]:
display(df_clientes)

id_cliente,nombre,edad,sexo,provincia,pais
10,Pablo Lopez,40,M,Sevilla,España
11,Luis Rodrigues,20,M,Bogota,Colombia
12,Alvaro Monroy,75,M,Monterrey,Mexico
13,Victor Ruiz,53,M,Valladolid,España
14,Melisa Aguado,30,F,Bogota,Colombia
15,Cristian Cuadrado,52,M,Guadalajara,Mexico
25,Ines Barcero,29,F,Guadalajara,Mexico
26,Celia Castro,47,F,Barcelona,España
27,Roberta Varado,64,F,Cali,Colombia
28,Walter Ramos,54,M,Cali,Colombia


In [0]:
df_consumos_diarios.show()

+----------+----------------+------------+----------------------+---------------------+----------+
|id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo|     fecha|
+----------+----------------+------------+----------------------+---------------------+----------+
|         1|             664|           3|                    25|                    0|2020-08-01|
|         1|             943|           0|                    12|                    5|2020-08-02|
|         1|            1035|           1|                    46|                   10|2020-08-03|
|         1|             760|           3|                    17|                    0|2020-08-04|
|         1|            1409|           1|                    31|                    4|2020-08-05|
|         1|            1627|           0|                    20|                    0|2020-08-06|
|         1|             739|           3|                    21|                    0|2020-08-07|
|         

In [0]:
df_consumos_diarios.show()

+----------+----------------+------------+----------------------+---------------------+----------+
|id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo|     fecha|
+----------+----------------+------------+----------------------+---------------------+----------+
|         1|             664|           3|                    25|                    0|2020-08-01|
|         1|             943|           0|                    12|                    5|2020-08-02|
|         1|            1035|           1|                    46|                   10|2020-08-03|
|         1|             760|           3|                    17|                    0|2020-08-04|
|         1|            1409|           1|                    31|                    4|2020-08-05|
|         1|            1627|           0|                    20|                    0|2020-08-06|
|         1|             739|           3|                    21|                    0|2020-08-07|
|         

In [0]:
df_consumos_diarios_join = df_consumos_diarios.select("id_cliente","minutos_llamadas_movil","minutos_llamadas_fijo")
df_consumos_diarios_join.show()

+----------+----------------------+---------------------+
|id_cliente|minutos_llamadas_movil|minutos_llamadas_fijo|
+----------+----------------------+---------------------+
|         1|                    25|                    0|
|         1|                    12|                    5|
|         1|                    46|                   10|
|         1|                    17|                    0|
|         1|                    31|                    4|
|         1|                    20|                    0|
|         1|                    21|                    0|
|         1|                    12|                    0|
|         1|                     0|                   24|
|         1|                    46|                    0|
|         1|                     0|                    0|
|         1|                    25|                    0|
|         1|                    41|                    0|
|         1|                    34|                  102|
|         1|  

In [0]:
# se procede a obtener  los clientes nuevos

clientes_nuevos_join = df_facturas_mes_actual.join(df_facturas_mes_ant,["id_cliente"],"leftanti")
clientes_nuevos_join.show()


+----------+---------+-------+----------+----+----------+
|id_cliente|id_oferta|importe|     fecha|edad|grupo_edad|
+----------+---------+-------+----------+----+----------+
|         4|        1|   70.0|2020-08-31|  32|         2|
|         9|        7|   78.5|2020-08-31|  24|         1|
|         9|       10|  29.99|2020-08-31|  24|         1|
|        19|        3|   85.0|2020-08-31|  38|         2|
|        21|        6|  124.5|2020-08-31|  29|         2|
|        28|        5|  102.5|2020-08-31|  54|         3|
+----------+---------+-------+----------+----+----------+



In [0]:
#clientes nuevos de este mes con 4 columnas: nombre_cliente_nuevo, edad, importe_total_mes_actual, total_minutos.
clientes_join = df_clientes_join= df_clientes.select("id_cliente","nombre")
clientes_join.show()

+----------+--------------------+
|id_cliente|              nombre|
+----------+--------------------+
|        10|         Pablo Lopez|
|        11|      Luis Rodrigues|
|        12|       Alvaro Monroy|
|        13|         Victor Ruiz|
|        14|       Melisa Aguado|
|        15|   Cristian Cuadrado|
|         7|       Carmen Arauzo|
|         8|    Silvia Rodriguez|
|         9|       David Cardoso|
|        19|Juan Carlos Iglesias|
|        20|          Oscar Rico|
|        21|       Fatima Cuevas|
|         4|         Pedro Conde|
|         5|          Ana Robles|
|         6|        David Roldan|
|         1|         Pablo Perez|
|         2|     Eduardo Redondo|
|         3|     Roberto Salazar|
|        16|       Cristina Sanz|
|        17|         Jorge Recio|
+----------+--------------------+
only showing top 20 rows



In [0]:
#nombre_cliente_nuevo, edad, importe_total_mes_actual, total_minutos.
clientes_nuevos_join = clientes_nuevos_join.join(clientes_join,["id_cliente"],"left")
clientes_nuevos_join = clientes_nuevos_join.withColumnRenamed("nombre", "nombre_cliente_nuevo")
clientes_nuevos_join = clientes_nuevos_join.join(df_consumos_diarios_join,["id_cliente"],"left")
clientes_nuevos_join.show()

+----------+---------+-------+----------+----+----------+--------------------+----------------------+---------------------+
|id_cliente|id_oferta|importe|     fecha|edad|grupo_edad|nombre_cliente_nuevo|minutos_llamadas_movil|minutos_llamadas_fijo|
+----------+---------+-------+----------+----+----------+--------------------+----------------------+---------------------+
|         4|        1|   70.0|2020-08-31|  32|         2|         Pedro Conde|                    19|                    0|
|         4|        1|   70.0|2020-08-31|  32|         2|         Pedro Conde|                    81|                    0|
|         4|        1|   70.0|2020-08-31|  32|         2|         Pedro Conde|                    81|                   20|
|         4|        1|   70.0|2020-08-31|  32|         2|         Pedro Conde|                    30|                    0|
|         4|        1|   70.0|2020-08-31|  32|         2|         Pedro Conde|                    62|                    0|
|       

In [0]:
# Verificar las columnas del DataFrame antes de la agrupación
clientes_nuevos_join.printSchema()

root
 |-- id_cliente: long (nullable = true)
 |-- id_oferta: long (nullable = true)
 |-- importe: double (nullable = true)
 |-- fecha: string (nullable = true)
 |-- edad: long (nullable = true)
 |-- grupo_edad: integer (nullable = true)
 |-- nombre_cliente_nuevo: string (nullable = true)
 |-- minutos_llamadas_movil: long (nullable = true)
 |-- minutos_llamadas_fijo: long (nullable = true)



In [0]:
# Agrupar y calcular totales
clientes_nuevos_join = clientes_nuevos_join.groupBy("id_cliente", "nombre_cliente_nuevo", "edad") \
    .agg(spark_sum("importe").alias("importe_total_mes_actual"),
        (spark_sum("minutos_llamadas_movil") + spark_sum("minutos_llamadas_fijo")).alias("total_minutos")
    ).orderBy("total_minutos")

# Mostrar el resultado
display(clientes_nuevos_join)

id_cliente,nombre_cliente_nuevo,edad,importe_total_mes_actual,total_minutos
28,Walter Ramos,54,3177.5,528
9,David Cardoso,24,3363.1899999999932,736
19,Juan Carlos Iglesias,38,2635.0,923
4,Pedro Conde,32,2170.0,1939
21,Fatima Cuevas,29,3859.5,2366


**8. Obtener un DF que contenga, para los clientes que ya existían en el mes anterior y siguen dados de alta este mes, 3 columnas: nombre, edad, n_dias_sin_sms. La última columna se refiere a obtener el número de veces que cada cliente NO ha enviado ningún SMS en el mes de Agosto. Forzar a que el comando "show" muestre 30 valores.** </br> (Si en todo el mes el cliente "A" no envío SMS 3 días en todo el mes, esta columna deberá contener el valor 3. Si hubiera algún cliente que hubiese enviado al menos 1 sms todos los dias del mes, tendrá valor 0 en esta columna).

In [0]:
# Se procede a llamar los dataframes necesarios, si se requiere algun otro, entonces se procedera agregar y realizar las transformaciones necesarias
df_consumos_diarios = spark.read.table("df_consumos_diarios")
df_clientes = spark.read.table("df_clientes")
df_consumos_diarios = spark.read.table("df_consumos_diarios")

In [0]:
df_consumos_diarios.show()

+----------+----------------+------------+----------------------+---------------------+----------+
|id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo|     fecha|
+----------+----------------+------------+----------------------+---------------------+----------+
|         1|             664|           3|                    25|                    0|2020-08-01|
|         1|             943|           0|                    12|                    5|2020-08-02|
|         1|            1035|           1|                    46|                   10|2020-08-03|
|         1|             760|           3|                    17|                    0|2020-08-04|
|         1|            1409|           1|                    31|                    4|2020-08-05|
|         1|            1627|           0|                    20|                    0|2020-08-06|
|         1|             739|           3|                    21|                    0|2020-08-07|
|         

In [0]:
df_clientes.show()

+----------+--------------------+----+----+-----------+--------+
|id_cliente|              nombre|edad|sexo|  provincia|    pais|
+----------+--------------------+----+----+-----------+--------+
|        10|         Pablo Lopez|  40|   M|    Sevilla|  España|
|        11|      Luis Rodrigues|  20|   M|     Bogota|Colombia|
|        12|       Alvaro Monroy|  75|   M|  Monterrey|  Mexico|
|        13|         Victor Ruiz|  53|   M| Valladolid|  España|
|        14|       Melisa Aguado|  30|   F|     Bogota|Colombia|
|        15|   Cristian Cuadrado|  52|   M|Guadalajara|  Mexico|
|         7|       Carmen Arauzo|  19|   F|   Medellin|Colombia|
|         8|    Silvia Rodriguez|  49|   F|  Barcelona|  España|
|         9|       David Cardoso|  24|   M|       Cali|Colombia|
|        19|Juan Carlos Iglesias|  38|   M|  Barcelona|  España|
|        20|          Oscar Rico|  22|   M| Valladolid|  España|
|        21|       Fatima Cuevas|  29|   F|       Cali|Colombia|
|         4|         Pedr

In [0]:
df_consumos_diarios.show()

+----------+----------------+------------+----------------------+---------------------+----------+
|id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo|     fecha|
+----------+----------------+------------+----------------------+---------------------+----------+
|         1|             664|           3|                    25|                    0|2020-08-01|
|         1|             943|           0|                    12|                    5|2020-08-02|
|         1|            1035|           1|                    46|                   10|2020-08-03|
|         1|             760|           3|                    17|                    0|2020-08-04|
|         1|            1409|           1|                    31|                    4|2020-08-05|
|         1|            1627|           0|                    20|                    0|2020-08-06|
|         1|             739|           3|                    21|                    0|2020-08-07|
|         

In [0]:
#DF que contenga, para los clientes que ya existían en el mes anterior y siguen dados de alta este mes, 3 columnas: nombre, edad, n_dias_sin_sms
df_facturas_mes_ant_clientes = df_facturas_mes_ant.select("id_cliente").join(df_clientes,["id_cliente"],"left")
df_clientes_activos = df_facturas_mes_ant_clientes.join(df_facturas_mes_actual.select("id_cliente"),["id_cliente"],"inner")
df_clientes_activos = df_clientes_activos.dropDuplicates(["id_cliente"])
df_clientes_activos.show()

+----------+-----------------+----+----+-----------+--------+
|id_cliente|           nombre|edad|sexo|  provincia|    pais|
+----------+-----------------+----+----+-----------+--------+
|         1|      Pablo Perez|  26|   M|     Madrid|  España|
|         2|  Eduardo Redondo|  58|   M|     Bogota|Colombia|
|         3|  Roberto Salazar|  68|   M|  Monterrey|  Mexico|
|         5|       Ana Robles|  41|   F| Valladolid|  España|
|         6|     David Roldan|  74|   M|Guadalajara|  Mexico|
|         7|    Carmen Arauzo|  19|   F|   Medellin|Colombia|
|         8| Silvia Rodriguez|  49|   F|  Barcelona|  España|
|        10|      Pablo Lopez|  40|   M|    Sevilla|  España|
|        11|   Luis Rodrigues|  20|   M|     Bogota|Colombia|
|        12|    Alvaro Monroy|  75|   M|  Monterrey|  Mexico|
|        13|      Victor Ruiz|  53|   M| Valladolid|  España|
|        14|    Melisa Aguado|  30|   F|     Bogota|Colombia|
|        15|Cristian Cuadrado|  52|   M|Guadalajara|  Mexico|
|       

In [0]:
df_clientes_activos = df_clientes_activos.select("id_cliente","nombre","edad")
df_clientes_activos.show()

+----------+-----------------+----+
|id_cliente|           nombre|edad|
+----------+-----------------+----+
|         1|      Pablo Perez|  26|
|         2|  Eduardo Redondo|  58|
|         3|  Roberto Salazar|  68|
|         5|       Ana Robles|  41|
|         6|     David Roldan|  74|
|         7|    Carmen Arauzo|  19|
|         8| Silvia Rodriguez|  49|
|        10|      Pablo Lopez|  40|
|        11|   Luis Rodrigues|  20|
|        12|    Alvaro Monroy|  75|
|        13|      Victor Ruiz|  53|
|        14|    Melisa Aguado|  30|
|        15|Cristian Cuadrado|  52|
|        17|      Jorge Recio|  18|
|        18|       Laura Luiz|  44|
|        22|     Clara Suarez|  21|
|        23|   Fernanda Gomez|  78|
|        25|     Ines Barcero|  29|
|        26|     Celia Castro|  47|
|        27|   Roberta Varado|  64|
+----------+-----------------+----+
only showing top 20 rows



**Una vez que tenemos el Df con los clientes que estan presentes en ambos meses, se procede a crear el  proceso para el calculo de los dias sin envio de sms**


In [0]:
# Crear una columna con la fecha en formato DateType
df_consumos_diarios = df_consumos_diarios.withColumn("fecha", to_date("fecha", "yyyy-MM-dd"))

# Calcular el número de días en el mes de agosto
num_days_in_august = 31

# est codido nos permite agrupar por cliente y contar el número de días que no se enviaron SMS
df_n_dias_sin_sms = df_consumos_diarios.groupBy("id_cliente").agg(
    (num_days_in_august - spark_sum(when(col("sms_enviados") > 0, 1).otherwise(0))).alias("n_dias_sin_sms")
)

# Unir con el DataFrame de clientes activos
df_resultado = df_clientes_activos.join(df_n_dias_sin_sms, ["id_cliente"], "left")

# Seleccionar y renombrar las columnas necesarias
df_resultado = df_resultado.select("nombre", "edad", "n_dias_sin_sms")

# Mostrar el resultado
df_resultado.show(30)

+-----------------+----+--------------+
|           nombre|edad|n_dias_sin_sms|
+-----------------+----+--------------+
|      Pablo Perez|  26|            14|
|  Eduardo Redondo|  58|             7|
|  Roberto Salazar|  68|             3|
|       Ana Robles|  41|             3|
|     David Roldan|  74|             1|
|    Carmen Arauzo|  19|            19|
| Silvia Rodriguez|  49|             2|
|      Pablo Lopez|  40|            11|
|   Luis Rodrigues|  20|            21|
|    Alvaro Monroy|  75|             3|
|      Victor Ruiz|  53|             7|
|    Melisa Aguado|  30|            20|
|Cristian Cuadrado|  52|             5|
|      Jorge Recio|  18|            22|
|       Laura Luiz|  44|             7|
|     Clara Suarez|  21|            26|
|   Fernanda Gomez|  78|            21|
|     Ines Barcero|  29|            13|
|     Celia Castro|  47|             0|
|   Roberta Varado|  64|             1|
|     Romina Verde|  29|            23|
|    Marta Rodrigo|  67|             2|


In [0]:
display(df_resultado) #con esto procedemos a mostar los datos del resultado de una mejor forma 

nombre,edad,n_dias_sin_sms
Pablo Perez,26,14
Eduardo Redondo,58,7
Roberto Salazar,68,3
Ana Robles,41,3
David Roldan,74,1
Carmen Arauzo,19,19
Silvia Rodriguez,49,2
Pablo Lopez,40,11
Luis Rodrigues,20,21
Alvaro Monroy,75,3


**9. Queremos obtener un Coeficiente de Ponderación que nos permita evaluar a cada cliente en función de su consumo para identificar los clientes más atractivos que forman parte de nuestra compañía. ESTE CÁLCULO SÓLO SE REALIZARÁ PARA LOS CLIENTES QUE TENGAN UNA SOLA OFERTA CONTRATADA CON LA COMPAÑÍA. </br> Este coeficiente se obtendrá en base a los consumos diarios, y por tanto sólo se tendrán en cuenta los clientes que existen en el mes de Agosto, ya que no tenemos datos de consumo del mes de Julio. </br> Las ponderaciones que se darán a cada uno de los consumos son las siguientes:** 
- 0.4 --> llamadas desde telefóno móvil.
- 0.3 --> datos_móviles_MB
- 0.2 --> llamadas desde teléfono fijo
- 0.1 --> SMS enviados </br> 

**Los pasos a seguir son los siguientes:**
  1. Obtener la suma de todos los días de cada uno de los 4 consumos para cada cliente.
  2. Obtener el máximo del cálculo anterior de todos los clientes para poder obtener un valor entre 0 y 1 para cada uno de los 4 consumos (recordar que sólo aplica para los clientes con UNA SOLA OFERTA CONTRATADA en el mes de Agosto. El cliente con mayor consumo de datos tendrá un valor de 1 en la columna 'datos_móviles_0_1')
  3. Multiplicar estas columnas obtenidas con valor entre 0 y 1 por su ponderación correspondiente (por ejemplo la columna "datos_moviles_0_1" se multiplicará por la ponderación de datos móviles 0.4).
  4. Por último se calculará la suma de las 4 ponderaciones obtenidas para calcular el Coeficiente de ponderación buscado (coeficiente_cliente), casteado a 3 decimales.
  5. Mostrar DF resultante ordenado por este coeficiente en sentido descendente, de manera que el primer registro corresponderá al cliente más atractivo. **El DF resultante tendrá 3 columnas: nombre, edad, coeficiente_cliente.**

In [0]:
# Se procede a llamar los dataframes necesarios, si se requiere algun otro, entonces se procedera agregar y realizar las transformaciones necesarias
df_consumos_diarios = spark.read.table("df_consumos_diarios")
df_clientes = spark.read.table("df_clientes")
df_facturas_mes_actual =  spark.read.table("df_facturas_mes_actual")

In [0]:
df_facturas_mes_actual.show()

+----------+---------+-------+----------+
|id_cliente|id_oferta|importe|     fecha|
+----------+---------+-------+----------+
|         1|        6|  124.5|2020-08-31|
|         1|        6|  118.6|2020-08-31|
|         2|        4|   95.0|2020-08-31|
|         3|        5|  102.5|2020-08-31|
|         4|        1|   70.0|2020-08-31|
|         5|        7|   80.5|2020-08-31|
|         5|        9|   40.5|2020-08-31|
|         6|        8|   84.0|2020-08-31|
|         7|        4|   95.0|2020-08-31|
|         8|        5|  100.5|2020-08-31|
|         9|        7|   78.5|2020-08-31|
|         9|       10|  29.99|2020-08-31|
|        18|        5|  102.5|2020-08-31|
|        18|        9|   39.0|2020-08-31|
|        19|        3|   85.0|2020-08-31|
|        21|        6|  124.5|2020-08-31|
|        14|        9|   42.5|2020-08-31|
|        15|        2|   78.5|2020-08-31|
|        15|        6|  115.0|2020-08-31|
|        17|        8|   84.0|2020-08-31|
+----------+---------+-------+----

In [0]:
df_clientes =df_clientes.select("id_cliente","nombre","edad")
df_clientes.show()

+----------+--------------------+----+
|id_cliente|              nombre|edad|
+----------+--------------------+----+
|        10|         Pablo Lopez|  40|
|        11|      Luis Rodrigues|  20|
|        12|       Alvaro Monroy|  75|
|        13|         Victor Ruiz|  53|
|        14|       Melisa Aguado|  30|
|        15|   Cristian Cuadrado|  52|
|         7|       Carmen Arauzo|  19|
|         8|    Silvia Rodriguez|  49|
|         9|       David Cardoso|  24|
|        19|Juan Carlos Iglesias|  38|
|        20|          Oscar Rico|  22|
|        21|       Fatima Cuevas|  29|
|         4|         Pedro Conde|  32|
|         5|          Ana Robles|  41|
|         6|        David Roldan|  74|
|         1|         Pablo Perez|  26|
|         2|     Eduardo Redondo|  58|
|         3|     Roberto Salazar|  68|
|        16|       Cristina Sanz|  49|
|        17|         Jorge Recio|  18|
+----------+--------------------+----+
only showing top 20 rows



In [0]:
df_facturas_mes_actual = df_facturas_mes_actual.join(df_clientes,["id_cliente"],"left").orderBy("id_cliente")
df_facturas_mes_actual.show()

+----------+---------+-------+----------+-----------------+----+
|id_cliente|id_oferta|importe|     fecha|           nombre|edad|
+----------+---------+-------+----------+-----------------+----+
|         1|        6|  124.5|2020-08-31|      Pablo Perez|  26|
|         1|        6|  118.6|2020-08-31|      Pablo Perez|  26|
|         2|        4|   95.0|2020-08-31|  Eduardo Redondo|  58|
|         3|        5|  102.5|2020-08-31|  Roberto Salazar|  68|
|         4|        1|   70.0|2020-08-31|      Pedro Conde|  32|
|         5|        7|   80.5|2020-08-31|       Ana Robles|  41|
|         5|        9|   40.5|2020-08-31|       Ana Robles|  41|
|         6|        8|   84.0|2020-08-31|     David Roldan|  74|
|         7|        4|   95.0|2020-08-31|    Carmen Arauzo|  19|
|         8|        5|  100.5|2020-08-31| Silvia Rodriguez|  49|
|         9|        7|   78.5|2020-08-31|    David Cardoso|  24|
|         9|       10|  29.99|2020-08-31|    David Cardoso|  24|
|        10|        6|  1

In [0]:
df_ofertas_por_cliente = df_facturas_mes_actual.groupBy("id_cliente", "nombre", "edad").agg(count("id_oferta").alias("num_ofertas"))
df_clientes_una_sola_oferta = df_ofertas_por_cliente.filter(col("num_ofertas") == 1)
display(df_clientes_una_sola_oferta)

id_cliente,nombre,edad,num_ofertas
28,Walter Ramos,54,1
29,Romina Verde,29,1
27,Roberta Varado,64,1
26,Celia Castro,47,1
2,Eduardo Redondo,58,1
3,Roberto Salazar,68,1
6,David Roldan,74,1
4,Pedro Conde,32,1
8,Silvia Rodriguez,49,1
7,Carmen Arauzo,19,1


**Obtener la suma de todos los días de cada uno de los 4 consumos para cada cliente.**

In [0]:
df_sumas_consumos = df_consumos_diarios.groupBy("id_cliente").agg(
    spark_sum("minutos_llamadas_movil").alias("total_minutos_llamadas_movil"),
    spark_sum("consumo_datos_MB").alias("total_datos_móviles_MB"),
    spark_sum("minutos_llamadas_fijo").alias("total_minutos_llamadas_fijo"),
    spark_sum("sms_enviados").alias("total_sms_enviados")
)

df_sumas_consumos.show()

+----------+----------------------------+----------------------+---------------------------+------------------+
|id_cliente|total_minutos_llamadas_movil|total_datos_móviles_MB|total_minutos_llamadas_fijo|total_sms_enviados|
+----------+----------------------------+----------------------+---------------------------+------------------+
|        29|                        1340|                 48922|                       1225|                14|
|        27|                         570|                  6512|                       1109|               233|
|        28|                         268|                 22768|                        260|               293|
|        30|                         326|                  9019|                       1574|               278|
|         7|                        1115|                 35204|                        255|                21|
|         6|                         277|                  3938|                       1529|            

In [0]:
df_sumas_consumos = df_sumas_consumos.join(df_clientes_una_sola_oferta,["id_cliente"],"inner")
display(df_sumas_consumos)

id_cliente,total_minutos_llamadas_movil,total_datos_móviles_MB,total_minutos_llamadas_fijo,total_sms_enviados,nombre,edad,num_ofertas
28,268,22768,260,293,Walter Ramos,54,1
29,1340,48922,1225,14,Romina Verde,29,1
27,570,6512,1109,233,Roberta Varado,64,1
26,1408,12915,1137,220,Celia Castro,47,1
2,1087,12605,759,111,Eduardo Redondo,58,1
3,689,5474,603,159,Roberto Salazar,68,1
6,277,3938,1529,275,David Roldan,74,1
4,1630,21061,309,27,Pedro Conde,32,1
8,1921,9755,1036,170,Silvia Rodriguez,49,1
7,1115,35204,255,21,Carmen Arauzo,19,1


In [0]:
max_consumos = df_sumas_consumos.agg(
    spark_sum("total_minutos_llamadas_movil").alias("max_minutos_llamadas_movil"),
    spark_sum("total_datos_móviles_MB").alias("max_datos_móviles_MB"),
    spark_sum("total_minutos_llamadas_fijo").alias("max_minutos_llamadas_fijo"),
    spark_sum("total_sms_enviados").alias("max_sms_enviados")
).collect()[0]

In [0]:
df_normalizado = df_sumas_consumos.withColumn("datos_móviles_0_1", col("total_datos_móviles_MB") / max_consumos["max_datos_móviles_MB"]
).withColumn("minutos_llamadas_movil_0_1", col("total_minutos_llamadas_movil") / max_consumos["max_minutos_llamadas_movil"]
).withColumn("minutos_llamadas_fijo_0_1", col("total_minutos_llamadas_fijo") / max_consumos["max_minutos_llamadas_fijo"]
).withColumn("sms_enviados_0_1", col("total_sms_enviados") / max_consumos["max_sms_enviados"])

In [0]:
# Aplicar ponderaciones
df_ponderado = df_normalizado.withColumn(
    "coeficiente_cliente",
    col("datos_móviles_0_1") * 0.3 +
    col("minutos_llamadas_movil_0_1") * 0.4 +
    col("minutos_llamadas_fijo_0_1") * 0.2 +
    col("sms_enviados_0_1") * 0.1
).select("id_cliente", "coeficiente_cliente")


In [0]:
# Unir con información de clientes para obtener nombre y edad
df_resultado = df_ponderado.join(df_clientes_una_sola_oferta, "id_cliente").select("nombre", "edad", "coeficiente_cliente")

# Ordenar por coeficiente en orden descendente
df_resultado = df_resultado.orderBy(col("coeficiente_cliente").desc())

# Mostrar el resultado
print("Esta es la tabla resumen de los cálculos de ponderación de los clientes:")
df_resultado.show()

Esta es la tabla resumen de los cálculos de ponderación de los clientes:
+--------------------+----+--------------------+
|              nombre|edad| coeficiente_cliente|
+--------------------+----+--------------------+
|        Romina Verde|  29| 0.08688029518271995|
|       Fatima Cuevas|  29| 0.08072401949500864|
|    Silvia Rodriguez|  49| 0.07869446236017838|
|        Celia Castro|  47| 0.07189382964587766|
|       Melisa Aguado|  30| 0.06485328127735929|
|         Pedro Conde|  32| 0.06183052296641726|
|       Carmen Arauzo|  19| 0.05777098521283136|
|     Eduardo Redondo|  58| 0.05277221146373834|
|      Fernanda Gomez|  78|0.047992504156016455|
|      Roberta Varado|  64|  0.0464478062305848|
|Juan Carlos Iglesias|  38|0.045502113581325575|
|        David Roldan|  74| 0.04537437482011635|
|         Jorge Recio|  18| 0.04172011704161756|
|        Walter Ramos|  54|0.040952896178247614|
|     Roberto Salazar|  68|0.037765200679635035|
|      Luis Rodrigues|  20| 0.036322246591284

**10. Queremos averiguar la fecha en la que entre los 3 clientes que más datos consumen de cada grupo_edad llegan a un consumo de 20 GB de datos móviles. En caso de que algún grupo_edad no llegue entre los 3 clientes a 20 GB en todo el mes, se asignará "null" como valor de esta columna (Recordar que 1 GB = 1024 MB). </br> Obtener un DF que contiene 4 registros, uno para cada grupo_edad y 3 columnas: grupo_edad, fecha_20_GB, datos_moviles_total_grupo_3_clientes.** (datos_moviles_total_grupo_3_clientes representa el total de datos consumidos en MB por los 3 clientes del grupo hasta final de mes)

In [0]:
# Se procede a llamar los dataframes necesarios, si se requiere algun otro, entonces se procedera agregar y realizar las transformaciones necesarias
df_consumos_diarios = spark.read.table("df_consumos_diarios")
df_clientes = spark.read.table("df_clientes")
df_facturas_mes_actual =  spark.read.table("df_facturas_mes_actual")

In [0]:
# crear los grupos de edad , tal y como se ha realizado anteriormente y en el caso de practica guiado
df_clientes_join= df_clientes.select("id_cliente","nombre", "edad")
df_clientes_join= df_clientes_join.withColumn("grupo_edad", when(col("edad") < 26, lit(1))
                                         .when(col("edad").between(26,40),lit(2))
                                         .when(col("edad").between(41,65),lit(3))
                                         .otherwise(lit(4)))
display(df_clientes_join)

id_cliente,nombre,edad,grupo_edad
10,Pablo Lopez,40,2
11,Luis Rodrigues,20,1
12,Alvaro Monroy,75,4
13,Victor Ruiz,53,3
14,Melisa Aguado,30,2
15,Cristian Cuadrado,52,3
25,Ines Barcero,29,2
26,Celia Castro,47,3
27,Roberta Varado,64,3
28,Walter Ramos,54,3


In [0]:
df_consumos_diarios_join = df_consumos_diarios.join(df_clientes_join,["id_cliente"],"left")
display(df_consumos_diarios_join)

id_cliente,consumo_datos_MB,sms_enviados,minutos_llamadas_movil,minutos_llamadas_fijo,fecha,nombre,edad,grupo_edad
27,249,6,24,24,2020-08-16,Roberta Varado,64,3
27,242,1,31,23,2020-08-17,Roberta Varado,64,3
27,237,15,15,57,2020-08-18,Roberta Varado,64,3
27,264,6,7,54,2020-08-19,Roberta Varado,64,3
27,255,1,41,19,2020-08-20,Roberta Varado,64,3
27,251,13,7,44,2020-08-21,Roberta Varado,64,3
27,281,2,30,34,2020-08-22,Roberta Varado,64,3
27,338,3,23,43,2020-08-23,Roberta Varado,64,3
27,201,13,8,57,2020-08-24,Roberta Varado,64,3
27,277,11,13,38,2020-08-25,Roberta Varado,64,3


In [0]:
# Convertir consumo de datos móviles a GB
df_consumos_diarios_join = df_consumos_diarios_join.withColumn("datos_móviles_GB", col("consumo_datos_MB") / 1024)
df_consumos_diarios_join.show()

+----------+----------------+------------+----------------------+---------------------+----------+--------------+----+----------+----------------+
|id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo|     fecha|        nombre|edad|grupo_edad|datos_móviles_GB|
+----------+----------------+------------+----------------------+---------------------+----------+--------------+----+----------+----------------+
|        27|             249|           6|                    24|                   24|2020-08-16|Roberta Varado|  64|         3|    0.2431640625|
|        27|             242|           1|                    31|                   23|2020-08-17|Roberta Varado|  64|         3|     0.236328125|
|        27|             237|          15|                    15|                   57|2020-08-18|Roberta Varado|  64|         3|    0.2314453125|
|        27|             264|           6|                     7|                   54|2020-08-19|Roberta Varado|  64|

In [0]:
# Una vez que tenemos los DF con los consumos en GB y con los grupos de edad, se procede a sacar los 3 clientes que más consumen por grupo de edad
window_spec = Window.partitionBy("grupo_edad").orderBy(col("datos_móviles_GB").desc())

df_consumos_diarios_join = df_consumos_diarios_join.withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= 3)

df_consumos_diarios_join.show()


+----------+----------------+------------+----------------------+---------------------+----------+-------------+----+----------+----------------+----+
|id_cliente|consumo_datos_MB|sms_enviados|minutos_llamadas_movil|minutos_llamadas_fijo|     fecha|       nombre|edad|grupo_edad|datos_móviles_GB|rank|
+----------+----------------+------------+----------------------+---------------------+----------+-------------+----+----------+----------------+----+
|        22|            2912|           0|                    61|                   30|2020-08-02| Clara Suarez|  21|         1|         2.84375|   1|
|        17|            2562|           0|                     0|                   18|2020-08-27|  Jorge Recio|  18|         1|     2.501953125|   2|
|        22|            2493|           0|                    20|                   46|2020-08-24| Clara Suarez|  21|         1|    2.4345703125|   3|
|        29|            2740|           0|                    28|                   46|2020-08

In [0]:

# Agregar el consumo total de datos móviles de los 3 clientes por fecha y grupo de edad
df_consumo_total = df_consumos_diarios_join.groupBy("grupo_edad", "fecha").agg(
    spark_sum("datos_móviles_GB").alias("datos_móviles_total_fecha")
)

# Encontrar la fecha en que el consumo total alcanza 20 GB
df_consumo_20_GB = df_consumo_total.groupBy("grupo_edad").agg(
    spark_max(when(col("datos_móviles_total_fecha") >= 20, col("fecha"))).alias("fecha_20_GB"),
    spark_sum("datos_móviles_total_fecha").alias("datos_móviles_total_grupo_3_clientes")
)

# Unir con el DataFrame original para obtener todos los grupos de edad
df_resultado_20gb = df_consumos_diarios_join.select("grupo_edad").distinct().join(df_consumo_20_GB, "grupo_edad", "left")

# Mostrar el resultado
df_resultado_20gb.show()

+----------+-----------+------------------------------------+
|grupo_edad|fecha_20_GB|datos_móviles_total_grupo_3_clientes|
+----------+-----------+------------------------------------+
|         1|       null|                        7.7802734375|
|         2|       null|                        7.2958984375|
|         3|       null|                        4.5263671875|
|         4|       null|                        1.6103515625|
+----------+-----------+------------------------------------+



## Conclusiones Generales##
Este ejercicio permite de forma introductoria pero a su vez de forma muy general cubrir transformaciones de datos y Data Frames que pueden encontrarse en el diario vivir de un analista de datos o Data Scientists, por  lo que cada una de las cuestiones o problemas de datos aqui analizado permiten al estudiante formarse en el uso de una herramienta somo lo es SPARK por medio de **DATABRICKS** en la modalidad cloud, brindando un ambiente adicional a la consola de PYTHON (local).

Este ejercicio como asignacion fue muy interesante y retador, seria interesante y necesario encontrar una serie de ejercicios con similitudes para seguir practicando las diversas funconalidades o posibilidades que se puede realizar con **DATABRICKS** y SPARK.

Muchas gracias

**Richard Douglas Grijalba**